In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np

### First, I have to define a function:

In [313]:
def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


## Scraping the data for the windparks from Wikipiedia;

### Baden-Würtemberg:

In [314]:
# URL der Wikipedia-Seite
bw_url = "https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Baden-Württemberg"

# Anfrage an die Webseite und HTML-Inhalt analysieren
response_bw = requests.get(bw_url)
soup_bw = BeautifulSoup(response_bw.text, 'html.parser')

# Tabelle finden
html_tables = soup_bw.find_all('table')
table_BW = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_BW.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)

        
#The reason I removed the crossed out windcrafts - because they don't work anymore, or for this moment       

In [315]:
len(filtered_rows)

200

In [316]:
# Let's print the first (the single) table and check its content
table_BW = html_tables[0]
print(table_BW)  

<table class="wikitable sortable zebra tabelle-kopf-fixiert" style="font-size:95%">
<tbody><tr bgcolor="#dfdfdf">
<th width="15%">Name
</th>
<th width="6%">Baujahr
</th>
<th width="4%">Gesamt-<br/>leistung (MW)
</th>
<th width="4%">Anzahl
</th>
<th width="20%">Typ (WKA)
</th>
<th width="10%">Ort
</th>
<th width="4%">Land-<br/>kreis
</th>
<th width="7%">Koordinaten
</th>
<th width="15%">Projektierer / Betreiber
</th>
<th width="15%">Bemerkungen
</th></tr>
<tr>
<td><a href="/wiki/Fraunhofer-Institut_f%C3%BCr_Chemische_Technologie" title="Fraunhofer-Institut für Chemische Technologie">Forschungswindkraftanlage RedoxWind</a><sup class="reference" id="cite_ref-4"><a href="#cite_note-4"><span class="cite-bracket">[</span>4<span class="cite-bracket">]</span></a></sup>
</td>
<td style="text-align:center">2017
</td>
<td style="text-align:center">2,0
</td>
<td style="text-align:center">1
</td>
<td>Qreon Q82 (1×)
</td>
<td style="text-align:center"><a href="/wiki/Karlsruhe" title="Karlsruhe">Karl

In [317]:
column_names = []

# Apply find_all() function with `th` element 
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

columns=table_BW.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
    

In [318]:
print(column_names)

['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [319]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}

In [320]:
#Just some cheks, for exmaple , the Name  in the first raw:

name=filtered_rows[1][0].strip()
name=re.sub(r'\[\d+\]', '', name)
name



'Forschungswindkraftanlage RedoxWind'

In [321]:
#Example year 

years=filtered_rows[3][1].strip()

commas=[3,7,11,15, 19, 23, 27,31]
output=''.join(c + ',' * (i in commas) for i, c in enumerate(years))


output

'1999,2013,'

In [322]:
#filling the dictionary
for x in filtered_rows[1:202]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    
df['Baujahr']

['2017,',
 '2003,',
 '1999,2013,',
 '1990,',
 '1992,',
 '2002,',
 '2001,',
 '2014,',
 '2020,',
 '2015,',
 '2001,',
 '2013,',
 '1998,',
 '2003,',
 '2001,',
 '1993,1995,',
 '1996,',
 '2005,',
 '2003,',
 '2000,',
 '2000,',
 '1999,',
 '2003,2024,',
 '2007,',
 '1994,1996,2015,',
 '2012,',
 '2023,',
 '2023,-202,4',
 '2023,-202,4',
 '1994,',
 '2003,',
 '1997,',
 '2002,',
 '2006,',
 '2012,',
 '2003,',
 '2020,',
 '2003,',
 '1997,1998,2013,',
 '1996,',
 '2006,',
 '2001,',
 '2011,',
 '2002,',
 '2022,',
 '1990,',
 '2005,',
 '1998,',
 '2004,',
 '2000,',
 '2024,',
 '1998,2005,',
 '2017,',
 '2004,2006,',
 '1997,2003,',
 '2003,',
 '2002,2003,',
 '2005,2010,',
 '2024,',
 '2023,',
 '2002,',
 '1998,2000,2003,2019,',
 '2020,',
 '2001,',
 '2015,',
 '1999,',
 '2000,2002,',
 '2003,',
 '2001,2002,',
 '2008,',
 '2000,',
 '2024,',
 '2000,2003,',
 '2005,',
 '2001,',
 '2001,',
 '2016,',
 '2017,',
 '2011,2016,2017,',
 '2000,2011,2023,',
 '2001,2002,2003,2018,2023,-202,4',
 '1998,2002,',
 '2017,2022,',
 '2002,2005,

In [323]:
#I decided to create additional column only for the first year 
df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]
df['Anfangjahr']

['2017',
 '2003',
 '1999',
 '1990',
 '1992',
 '2002',
 '2001',
 '2014',
 '2020',
 '2015',
 '2001',
 '2013',
 '1998',
 '2003',
 '2001',
 '1993',
 '1996',
 '2005',
 '2003',
 '2000',
 '2000',
 '1999',
 '2003',
 '2007',
 '1994',
 '2012',
 '2023',
 '2023',
 '2023',
 '1994',
 '2003',
 '1997',
 '2002',
 '2006',
 '2012',
 '2003',
 '2020',
 '2003',
 '1997',
 '1996',
 '2006',
 '2001',
 '2011',
 '2002',
 '2022',
 '1990',
 '2005',
 '1998',
 '2004',
 '2000',
 '2024',
 '1998',
 '2017',
 '2004',
 '1997',
 '2003',
 '2002',
 '2005',
 '2024',
 '2023',
 '2002',
 '1998',
 '2020',
 '2001',
 '2015',
 '1999',
 '2000',
 '2003',
 '2001',
 '2008',
 '2000',
 '2024',
 '2000',
 '2005',
 '2001',
 '2001',
 '2016',
 '2017',
 '2011',
 '2000',
 '2001',
 '1998',
 '2017',
 '2002',
 '2000',
 '2007',
 '2019',
 '2002',
 '2017',
 '2018',
 '2001',
 '2000',
 '2004',
 '2014',
 '1997',
 '2016',
 '2017',
 '2021',
 '2016',
 '2024',
 '2016',
 '2017',
 '2000',
 '2000',
 '2016',
 '2016',
 '2000',
 '2021',
 '2017',
 '2004',
 '2001',
 

In [324]:
#filling the dictionaries for the remaining columns:

for i in filtered_rows [1:204]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
        
    
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())

    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  
    

In [325]:
df['Name']

['Forschungswindkraftanlage RedoxWind',
 'Windkraftanlage Alexanderschanze',
 'Windkraftanlage Alpirsbach',
 'Windkraftanlage Auenberg',
 'Windkraftanlage auf dem Hohwartsattel',
 'Windkraftanlage Bad Mergentheim',
 'Windkraftanlage Binselberg',
 'Windkraftanlage Birkenzell',
 'Windkraftanlage Bopfingen',
 'Windkraftanlage Bronn',
 'Windkraftanlage Dauchingen',
 'Windkraftanlage Dettenberg',
 'Windkraftanlage Dewangen',
 'Windkraftanlage Dornhan',
 'Windkraftanlage Ellenberg',
 'Windkraftanlage Emmingen',
 'Windkraftanlage Fernhöhe',
 'Windkraftanlage Fröhnd',
 'Windkraftanlage Fürstenberg',
 'Windkraftanlage Glatten',
 'Windkraftanlage Grüner Heiner',
 'Windkraftanlage Herbolzheim',
 'Windkraftanlage Holzschlägermatte',
 'Windkraftanlage Honhardt',
 'Windkraftanlage Hornisgrinde',
 'Windkraftanlage Ingersheim',
 'Windkraftanlage Kallenwald',
 'Windkraftanlagen Königseiche',
 'Windkraftanlagen Laichingen-Weidstetten',
 'Windkraftanlage Leibertingen',
 'Windkraftanlage Lenzkirch-Kopp',


In [335]:
df_bw= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })
df_bw.head(3)

,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr
0,Forschungswindkraftanlage RedoxWind,"2017,","2,0",1,Qreon Q82 (1×),Karlsruhe,KA,"49° 1′ 9″ N, 8° 31′ 4″ O",Fraunhofer ICT,Forschungsprojekt mit kombiniertem Batteriespe...,2017
1,Windkraftanlage Alexanderschanze,"2003,","1,5",1,Südwind S70 (1×),Bad Peterstal-Griesbach,OG,"48° 28′ 22″ N, 8° 16′ 7″ O","Schmalz, Windenergieanlage „Bei der Schanz“",errichtet an der Schwarzwaldhochstraße im Bere...,2003
2,Windkraftanlage Alpirsbach,"1999,2013,","2,3",1,Enercon E-70 E4 (1×),Römlinsdorf,FDS,"48° 20′ 0″ N, 8° 28′ 34″ O","Windkraft Römlinsdorf, Stadtwerke Stuttgart",Repowering 2013 (1× Enercon E-70 E4 statt 1× F...,1999


In [336]:
df_bw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      199 non-null    object
 1   Baujahr                   199 non-null    object
 2   Gesamtleistung (MW)       199 non-null    object
 3   Anzahl                    199 non-null    object
 4   Typ (WKA)                 199 non-null    object
 5   Ort                       199 non-null    object
 6   Landkreis                 199 non-null    object
 7   Koordinaten               199 non-null    object
 8   Projektierer / Betreiber  199 non-null    object
 9   Bemerkungen               199 non-null    object
 10  Anfangjahr                199 non-null    object
dtypes: object(11)
memory usage: 17.2+ KB


In [337]:
#we have to do it because in wikipedia park for this windpark
#the coordinates are absent
df_bw['Koordinaten'].loc[df_bw['Name'].str.contains('Windpark Hohenlochen')]= str('♁48° 17′ N, 8° 11′ O')


In [338]:
#we make two variable from the 'Koordinaten' - 'Latitude' and 'Longitude'
fn = lambda x: pd.Series([i for i in (x.split(','))])
pad = df_bw['Koordinaten'].apply(fn)

pad.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_bw['Latitude']=pad['Latitude']
df_bw['Longitude']=pad['Longitude']
df_bw['Land']='BW'

In [339]:
#I have to chane the dtypes of 'Anzahl' and 'Gesamtleistung'
df_bw['Anzahl'] = [int(str(i)) for i in df_bw['Anzahl']]
df_bw['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_bw['Gesamtleistung (MW)']]

df_bw.head(3)

,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Forschungswindkraftanlage RedoxWind,"2017,",2.0,1,Qreon Q82 (1×),Karlsruhe,KA,"49° 1′ 9″ N, 8° 31′ 4″ O",Fraunhofer ICT,Forschungsprojekt mit kombiniertem Batteriespe...,2017,49° 1′ 9″ N,8° 31′ 4″ O,BW
1,Windkraftanlage Alexanderschanze,"2003,",1.5,1,Südwind S70 (1×),Bad Peterstal-Griesbach,OG,"48° 28′ 22″ N, 8° 16′ 7″ O","Schmalz, Windenergieanlage „Bei der Schanz“",errichtet an der Schwarzwaldhochstraße im Bere...,2003,48° 28′ 22″ N,8° 16′ 7″ O,BW
2,Windkraftanlage Alpirsbach,"1999,2013,",2.3,1,Enercon E-70 E4 (1×),Römlinsdorf,FDS,"48° 20′ 0″ N, 8° 28′ 34″ O","Windkraft Römlinsdorf, Stadtwerke Stuttgart",Repowering 2013 (1× Enercon E-70 E4 statt 1× F...,1999,48° 20′ 0″ N,8° 28′ 34″ O,BW


In [340]:
df_bw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name                      199 non-null    object 
 1   Baujahr                   199 non-null    object 
 2   Gesamtleistung (MW)       199 non-null    float64
 3   Anzahl                    199 non-null    int64  
 4   Typ (WKA)                 199 non-null    object 
 5   Ort                       199 non-null    object 
 6   Landkreis                 199 non-null    object 
 7   Koordinaten               199 non-null    object 
 8   Projektierer / Betreiber  199 non-null    object 
 9   Bemerkungen               199 non-null    object 
 10  Anfangjahr                199 non-null    object 
 11  Latitude                  199 non-null    object 
 12  Longitude                 199 non-null    object 
 13  Land                      199 non-null    object 
dtypes: float64

In [341]:
orts_bw=df_bw.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)

orts_bw.head(10)

#we see that some Land-Kreis are mixture of two :GPAA and HNKÜN
#if we use 'land kreis' later, we should something do with this
#to replace such values with one of the Land kreises (research in the internet)
#but if I am going to use only coordinates I can leave it as it is so far

,Anzahl
Landkreis,
TBB,149
SHA,113
AA,84
OG,47
UL,46
MOS,43
GP,41
HDH,38
GPAA,19


In [342]:
orts_bw_MW=df_bw.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)

orts_bw_MW.head(10)


,Gesamtleistung (MW)
Landkreis,
SHA,350.420
TBB,303.040
AA,211.040
OG,140.285
MOS,121.460
HDH,113.430
GP,80.600
UL,68.140
HNKÜN,54.900


In [343]:
bw=df_bw[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
bw.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Forschungswindkraftanlage RedoxWind,"2017,",2017,2.0,1,Qreon Q82 (1×),Karlsruhe,KA,Fraunhofer ICT,Forschungsprojekt mit kombiniertem Batteriespe...,49° 1′ 9″ N,8° 31′ 4″ O,BW
1,Windkraftanlage Alexanderschanze,"2003,",2003,1.5,1,Südwind S70 (1×),Bad Peterstal-Griesbach,OG,"Schmalz, Windenergieanlage „Bei der Schanz“",errichtet an der Schwarzwaldhochstraße im Bere...,48° 28′ 22″ N,8° 16′ 7″ O,BW
2,Windkraftanlage Alpirsbach,"1999,2013,",1999,2.3,1,Enercon E-70 E4 (1×),Römlinsdorf,FDS,"Windkraft Römlinsdorf, Stadtwerke Stuttgart",Repowering 2013 (1× Enercon E-70 E4 statt 1× F...,48° 20′ 0″ N,8° 28′ 34″ O,BW


### Approach for such shared between two or three 'Landkreises'Identify the Primary District: Determine if one of the Landkreise has a larger proportion of the wind park. The primary district might have more influence on regulatory and environmental decisions if the majority of the park's area falls within its boundaries. - I consider, that primary Landkries would the first in the list, I  checked this through Wikipedia data, and it seems to be a case

In [344]:
print(bw['Landkreis'].unique())

dict_bw={'AAHDH':'AA',
        'RWOG':'RW',
        'HNKÜN':'HN',
        'GPAA':'GP',
        'CWFDS':'CW',
        'FRFR':'FR',
        'FREM':'FR',
        'OGRWVS':'OG',
        'TBBMOS':'TBB'}

bw['Landkreis'].replace(dict_bw, inplace=True)
print(bw['Landkreis'].unique())



['KA' 'OG' 'FDS' 'VS' 'FR' 'TBB' 'SHA' 'AA' 'BC' 'RW' 'CW' 'LÖ' 'S' 'EM'
 'LB' 'GP' 'UL' 'SIG' 'KÜN' 'HDH' 'HN' 'WN' 'WT' 'MOS' 'RV' 'TUT' 'RT'
 'AAHDH' 'RWOG' 'HNKÜN' 'BL' 'GPAA' 'CWFDS' 'TBBMOS' 'FRFR' 'FREM' 'PF'
 'OGRWVS' 'KN']
['KA' 'OG' 'FDS' 'VS' 'FR' 'TBB' 'SHA' 'AA' 'BC' 'RW' 'CW' 'LÖ' 'S' 'EM'
 'LB' 'GP' 'UL' 'SIG' 'KÜN' 'HDH' 'HN' 'WN' 'WT' 'MOS' 'RV' 'TUT' 'RT'
 'BL' 'PF' 'KN']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/3646264693.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bw['Landkreis'].replace(dict_bw, inplace=True)


### Bayern:

In [345]:
#Bayerische windaprks
url_bayern="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Bayern"
# use requests.get() method with the provided static_url
response_bayern=requests.get(url_bayern).text
# assign the response to a object
soup_bayern=BeautifulSoup(response_bayern, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_bayern.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_bayern.find_all('table')
table_bayern = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_bayern.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Bayern is:', len(filtered_rows))


The title is:
Liste von Windkraftanlagen in Bayern – Wikipedia
The number of windparks in Bayern is: 349


In [346]:
tml_tables=soup_bayern.find_all('table')
table_bayern = html_tables[0]

column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

columns=table_bayern.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 
print('We can see that these are the same columns as for BW')


['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']
We can see that these are the same columns as for BW


In [347]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:353]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:353]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_bayern= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })

In [348]:
print(df_bayern.info())

df_bayern['Land']='Bayern'

df_bayern['Anzahl']=df_bayern['Anzahl'].fillna(0)
df_bayern['Anzahl'] = [int(str(i)) for i in df_bayern['Anzahl']]
df_bayern['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_bayern['Gesamtleistung (MW)']]

fn = lambda x: pd.Series([i for i in (x.split(','))])
pad = df_bayern['Koordinaten'].apply(fn)

pad.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_bayern['Latitude']=pad['Latitude']
df_bayern['Longitude']=pad['Longitude']

df_bayern.head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      348 non-null    object
 1   Baujahr                   348 non-null    object
 2   Gesamtleistung (MW)       348 non-null    object
 3   Anzahl                    348 non-null    object
 4   Typ (WKA)                 348 non-null    object
 5   Ort                       348 non-null    object
 6   Landkreis                 348 non-null    object
 7   Koordinaten               348 non-null    object
 8   Projektierer / Betreiber  348 non-null    object
 9   Bemerkungen               348 non-null    object
 10  Anfangjahr                348 non-null    object
dtypes: object(11)
memory usage: 30.0+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Land,Latitude,Longitude
0,Windkraftanlage Alfeld-Waller,"2009,",2.00,1,Vestas V90-2.0MW (1×),Waller,LAU,"49° 25′ 26″ N, 11° 29′ 21″ O","ATM Hartmann, SRE Sachs",,2009,Bayern,49° 25′ 26″ N,11° 29′ 21″ O
1,Windkraftanlage Allersberg,"2002,",0.60,1,Enercon E-40/6.44 (1×),Göggelsbuch,RH,"49° 12′ 56″ N, 11° 12′ 14″ O",,,2002,Bayern,49° 12′ 56″ N,11° 12′ 14″ O
2,Windkraftanlage Altmannstein,"2013,",1.80,1,Vestas V100-1.8MW (1×),Altmannstein,EI,"48° 56′ 25″ N, 11° 34′ 54″ O",Windkraft Kernl,,2013,Bayern,48° 56′ 25″ N,11° 34′ 54″ O
3,Windkraftanlage Ammerfeld,"2009,",2.00,1,Enercon E-82 (1×),Ammerfeld,ND,"48° 47′ 26″ N, 10° 59′ 4″ O",Bavaria Windkraft,Bei Inbetriebnahme größte Windkraftanlage in B...,2009,Bayern,48° 47′ 26″ N,10° 59′ 4″ O
4,Windkraftanlage Ansbach-Strüth,"2020,–202,1",3.63,1,GE Wind Energy 3.6-137 (1×),Ansbach,AN,"49° 20′ 3″ N, 10° 33′ 46″ O",WP Strüth GmbH,,2020,Bayern,49° 20′ 3″ N,10° 33′ 46″ O
5,Windkraftanlage Baisweil,"2018,",3.60,1,Senvion 3.6M140 (1×),Baisweil,OAL,"47° 57′ 1″ N, 10° 31′ 19″ O",,,2018,Bayern,47° 57′ 1″ N,10° 31′ 19″ O
6,Windkraftanlage Batzhausen,"2001,",0.60,1,Pfleiderer PWE 650 (1×),Batzhausen,NM,"49° 11′ 25″ N, 11° 37′ 46″ O",Jura Energy,,2001,Bayern,49° 11′ 25″ N,11° 37′ 46″ O
7,Windkraftanlage Beratzhausen-Buxlohe,"2006,",0.80,1,Enercon E-48 (1×),Buxlohe,R,"49° 7′ 7″ N, 11° 49′ 47″ O",,,2006,Bayern,49° 7′ 7″ N,11° 49′ 47″ O
8,Windkraftanlage Bieberehren-Klingen,"2006,",1.50,1,Vestas V82-1.5MW (1×),Klingen,WÜ,"49° 29′ 7″ N, 9° 59′ 7″ O",ATM,,2006,Bayern,49° 29′ 7″ N,9° 59′ 7″ O
9,Windkraftanlage Böhen,"2002,",0.60,1,Enercon E-40/6.44 (1×),Böhen,MN,"47° 53′ 3″ N, 10° 17′ 24″ O",,,2002,Bayern,47° 53′ 3″ N,10° 17′ 24″ O


In [349]:
orts_bayern=df_bayern.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_bayern

,Anzahl
Landkreis,
HO,109
AN,72
WÜ,67
NM,63
NEA,55
...,...
CHA,1
NU,1
FRG,1


In [350]:
orts_bayern_MW=df_bayern.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_bayern_MW




,Gesamtleistung (MW)
Landkreis,
HO,242.70
AN,171.68
NM,149.00
WÜ,138.15
NEA,138.05
...,...
PA,0.60
WM,0.60
FRG,0.20


In [351]:
bayern=df_bayern[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
bayern.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Alfeld-Waller,"2009,",2009,2.0,1,Vestas V90-2.0MW (1×),Waller,LAU,"ATM Hartmann, SRE Sachs",,49° 25′ 26″ N,11° 29′ 21″ O,Bayern
1,Windkraftanlage Allersberg,"2002,",2002,0.6,1,Enercon E-40/6.44 (1×),Göggelsbuch,RH,,,49° 12′ 56″ N,11° 12′ 14″ O,Bayern
2,Windkraftanlage Altmannstein,"2013,",2013,1.8,1,Vestas V100-1.8MW (1×),Altmannstein,EI,Windkraft Kernl,,48° 56′ 25″ N,11° 34′ 54″ O,Bayern


In [444]:
print(bayern['Landkreis'].unique())

dict_ba={'LAUNM':'LAU',
        'ANNEA':'AN',
        'LIFKU':'LIF',
        'BTKU':'BT',
        'EIRH':'EI',
        'ANWUG':'AN',
        'EIWUG':'EI',
        'ASNEW':'AS',
        'MSPSW':'MSP',
        'KCKU':'KC',
        'MSPWÜ':'MSP',
        'AN\nAN':'AN',
        'BALIF':'BA',
        'OALOA':'OAL',
        'NEAERH':'NEA',
        'GZA':'GZ',
        'EG':'PAN', #Eventuell, change to PAN
        'HOBT':'HO'}

bayern['Landkreis'].replace(dict_ba, inplace=True)
print(bayern['Landkreis'].unique())



['LAU' 'RH' 'EI' 'ND' 'AN' 'OAL' 'NM' 'R' 'WÜ' 'MN' 'REG' 'KU' 'KT' 'DAH'
 'MSP' 'BT' 'M' 'EBE' 'FS' 'PAN' 'AS' 'PAF' 'DLG' 'FÜ' 'OA' 'SW' 'LA'
 'FRG' 'CHA' 'FO' 'EG' 'KC' 'NEA' 'FFB' 'SAD' 'WUG' 'ERH' 'TIR' 'WM' 'KEH'
 'RO' 'BA' 'TS' 'NU' 'HO' 'PA' 'WUN' 'DON' 'AIC' 'HAS' 'LL' 'DGF' 'A'
 'MIL' 'NEW' 'KG' 'NES' 'LIF' 'CO' 'MB' 'GZ' 'SR' 'STA']
['LAU' 'RH' 'EI' 'ND' 'AN' 'OAL' 'NM' 'R' 'WÜ' 'MN' 'REG' 'KU' 'KT' 'DAH'
 'MSP' 'BT' 'M' 'EBE' 'FS' 'PAN' 'AS' 'PAF' 'DLG' 'FÜ' 'OA' 'SW' 'LA'
 'FRG' 'CHA' 'FO' 'KC' 'NEA' 'FFB' 'SAD' 'WUG' 'ERH' 'TIR' 'WM' 'KEH' 'RO'
 'BA' 'TS' 'NU' 'HO' 'PA' 'WUN' 'DON' 'AIC' 'HAS' 'LL' 'DGF' 'A' 'MIL'
 'NEW' 'KG' 'NES' 'LIF' 'CO' 'MB' 'GZ' 'SR' 'STA']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/1935588164.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bayern['Landkreis'].replace(dict_ba, inplace=True)


### Berlin and Brandenburg:

In [353]:
url_be_ba="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Berlin_und_Brandenburg"
# use requests.get() method with the provided static_url
# use requests.get() method with the provided static_url
response_be_ba=requests.get(url_be_ba).text
# assign the response to a object
soup_be_ba=BeautifulSoup(response_be_ba, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_be_ba.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_be_ba.find_all('table')
table_be_ba = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_be_ba.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Berlin/Brandenburg is:', len(filtered_rows))


column_names = []
columns=table_be_ba.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 
print('We can see that these are the same columns as for BW')


The title is:
Liste von Windkraftanlagen in Berlin und Brandenburg – Wikipedia
The number of windparks in Berlin/Brandenburg is: 291
['Name', 'Baujahr', 'Gesamt-  Leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land-  kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']
We can see that these are the same columns as for BW


In [354]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:353]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:353]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_berlin_br= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })


In [355]:
print(df_berlin_br.info())

df_berlin_br['Land']='Ber Br'

df_berlin_br['Anzahl']=df_berlin_br['Anzahl'].fillna(0)
df_berlin_br['Anzahl'] = [int(str(i)) for i in df_berlin_br['Anzahl']]
df_berlin_br['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_berlin_br['Gesamtleistung (MW)']]

fn = lambda x: pd.Series([i for i in (x.split(','))])
pad = df_berlin_br['Koordinaten'].apply(fn)

pad.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_berlin_br['Latitude']=pad['Latitude']
df_berlin_br['Longitude']=pad['Longitude']

df_berlin_br.head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      290 non-null    object
 1   Baujahr                   290 non-null    object
 2   Gesamtleistung (MW)       290 non-null    object
 3   Anzahl                    290 non-null    object
 4   Typ (WKA)                 290 non-null    object
 5   Ort                       290 non-null    object
 6   Landkreis                 290 non-null    object
 7   Koordinaten               290 non-null    object
 8   Projektierer / Betreiber  290 non-null    object
 9   Bemerkungen               290 non-null    object
 10  Anfangjahr                290 non-null    object
dtypes: object(11)
memory usage: 25.1+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Land,Latitude,Longitude
0,I. Windkraft­anlage Pankow,"2008,",2.00,1,Enercon E-82 (1×),Buch,B,"52° 38′ 10″ N, 13° 25′ 47″ O",NEB,bis Ende 2014 einzige Windkraftanlage in Berli...,2008,Ber Br,52° 38′ 10″ N,13° 25′ 47″ O
1,II. und VI. Wind­kraft­anlage Pankow,"2014, 202,1",6.50,2,Enercon E-82 E2 (1×) Enercon E-138 EP3 E2 (1×),Stadtrand­siedlung Malchow,B,"52° 35′ 29″ N, 13° 30′ 4″ O",NEB,,2014,Ber Br,52° 35′ 29″ N,13° 30′ 4″ O
2,III. und IV. Wind­kraft­anlage Pankow,"2015,",4.70,2,Enercon E-92 (2×),Französisch Buchholz,B,"52° 37′ 28″ N, 13° 26′ 23″ O",Phase I GmbH & Co Windkraft Pankow,auf dem Gelände von Heron Fireworks,2015,Ber Br,52° 37′ 28″ N,13° 26′ 23″ O
3,V. Windkraft­anlage Pankow,"2016,",3.40,1,Senvion 3.4M104 (1×),Buch,B,"52° 38′ 27″ N, 13° 26′ 6″ O",WEA Pankow GmbH,auf dem Gelände von Dare GmbH,2016,Ber Br,52° 38′ 27″ N,13° 26′ 6″ O
4,Windkraftanlage Banzendorf,"1999,",0.50,1,Enercon E-40/5.40 (1×),Banzendorf,OPR,"53° 2′ 27″ N, 13° 1′ 9″ O",Windenergie Wenger-Rosenau GmbH & Co. KG; Ener...,,1999,Ber Br,53° 2′ 27″ N,13° 1′ 9″ O
5,Windkraftanlage Berge,"1994,",0.05,1,Krogmann 15/50 (1×),Berge,HVL,"52° 36′ 55″ N, 12° 47′ 1″ O",Jugendhof Brandenburg e. V.,"seit 2021 vorübergehend stillgelegt, soll Ende...",1994,Ber Br,52° 36′ 55″ N,12° 47′ 1″ O
6,Windkraftanlage Bärenbrück (Windkraftanlage Kr...,"1994,",0.50,1,Enercon E-40/5.40 (1×),Bärenbrück,SPN,"51° 48′ 25″ N, 14° 27′ 20″ O",Boreas,erste Windkraftanlage in der Lausitz,1994,Ber Br,51° 48′ 25″ N,14° 27′ 20″ O
7,Windkraft­anlage Booßen,"2002,",0.60,1,Enercon E-40/6.44 (1×),Booßen,FF,"52° 22′ 37″ N, 14° 27′ 47″ O",,,2002,Ber Br,52° 22′ 37″ N,14° 27′ 47″ O
8,Windkraftanlage Diedersdorf,"1996,",0.60,1,Vestas V44-600kW (1×),Diedersdorf,MOL,"52° 31′ 15″ N, 14° 21′ 16″ O",,,1996,Ber Br,52° 31′ 15″ N,14° 21′ 16″ O
9,Windkraft­anlage Euro­Speedway Lausitz,"2012,",7.60,1,Enercon E-126 EP8 (1×),Hörlitz,OSL,"51° 32′ 7″ N, 13° 55′ 17″ O",Energiequelle GmbH,,2012,Ber Br,51° 32′ 7″ N,13° 55′ 17″ O


In [356]:
orts_berlin_br=df_berlin_br.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_berlin_br.head(5)

,Anzahl
Landkreis,
UM,666
PR,567
OPR,315
EE,303
MOL,294


In [357]:
orts_berlin_br_MW=df_berlin_br.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)

orts_berlin_br_MW

,Gesamtleistung (MW)
Landkreis,
UM,1719.41
PR,1232.41
MOL,707.08
LDS,616.40
EE,613.50
TF,563.05
OPR,497.17
LOS,466.17
HVL,361.50


In [358]:
br=df_berlin_br[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
br.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,I. Windkraft­anlage Pankow,"2008,",2008,2.0,1,Enercon E-82 (1×),Buch,B,NEB,bis Ende 2014 einzige Windkraftanlage in Berli...,52° 38′ 10″ N,13° 25′ 47″ O,Ber Br
1,II. und VI. Wind­kraft­anlage Pankow,"2014, 202,1",2014,6.5,2,Enercon E-82 E2 (1×) Enercon E-138 EP3 E2 (1×),Stadtrand­siedlung Malchow,B,NEB,,52° 35′ 29″ N,13° 30′ 4″ O,Ber Br
2,III. und IV. Wind­kraft­anlage Pankow,"2015,",2015,4.7,2,Enercon E-92 (2×),Französisch Buchholz,B,Phase I GmbH & Co Windkraft Pankow,auf dem Gelände von Heron Fireworks,52° 37′ 28″ N,13° 26′ 23″ O,Ber Br


In [359]:
print(br['Landkreis'].unique())

dict_br={'TF PM':'TF',
        'PM TF':'PM',
        'TF LDS':'TF',
        'BAR MOL':'BAR',
        'OSL EE':'OSL',
        'OPR PR':'OPR',
        'BAR UM':'BAR',
        'SPN OSL':'SPN',
        'LDS LOS':'LDS',
        'OSL SPN':'OSL'}

br['Landkreis'].replace(dict_br, inplace=True)
print(br['Landkreis'].unique())

['B' 'OPR' 'HVL' 'SPN' 'FF' 'MOL' 'OSL' 'PR' 'TF' 'UM' 'LOS' 'PM' 'LDS'
 'BAR' 'EE' 'BRB' 'TF PM' 'OHV' 'CB' 'TF LDS' 'PM TF' 'BAR MOL' 'OSL EE'
 'OPR PR' 'BAR UM' 'SPN OSL' 'LDS LOS' 'OSL SPN']
['B' 'OPR' 'HVL' 'SPN' 'FF' 'MOL' 'OSL' 'PR' 'TF' 'UM' 'LOS' 'PM' 'LDS'
 'BAR' 'EE' 'BRB' 'OHV' 'CB']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/3203025698.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  br['Landkreis'].replace(dict_br, inplace=True)


### Bremen, Hamburg, Niedersachsen:


In [360]:
url_nord="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Bremen,_Hamburg_und_Niedersachsen"
# use requests.get() method with the provided static_url
response_nord=requests.get(url_nord).text
# assign the response to a object
soup_nord=BeautifulSoup(response_nord, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_nord.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_nord.find_all('table')
table_nord = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_nord.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Bremen, Hamburg, Niedersachsen is:', len(filtered_rows))



column_names = []
columns=table_nord.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 
print('We can see that these are the same columns as for BW')

The title is:
Liste von Windkraftanlagen in Bremen, Hamburg und Niedersachsen – Wikipedia
The number of windparks in Bremen, Hamburg, Niedersachsen is: 972
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']
We can see that these are the same columns as for BW


In [361]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_nord= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })


In [362]:
print(df_nord.info())

df_nord['Land']='HB,HH, H'

df_nord['Anzahl']=df_nord['Anzahl'].fillna(0)
df_nord['Anzahl'] = [int(str(i)) for i in df_nord['Anzahl']]
df_nord['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_nord['Gesamtleistung (MW)']]

fn = lambda x: pd.Series([i for i in (x.split(','))])
pad = df_nord['Koordinaten'].apply(fn)

pad.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_nord['Latitude']=pad['Latitude']
df_nord['Longitude']=pad['Longitude']

df_nord.head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971 entries, 0 to 970
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      971 non-null    object
 1   Baujahr                   971 non-null    object
 2   Gesamtleistung (MW)       971 non-null    object
 3   Anzahl                    971 non-null    object
 4   Typ (WKA)                 971 non-null    object
 5   Ort                       971 non-null    object
 6   Landkreis                 971 non-null    object
 7   Koordinaten               971 non-null    object
 8   Projektierer / Betreiber  971 non-null    object
 9   Bemerkungen               971 non-null    object
 10  Anfangjahr                971 non-null    object
dtypes: object(11)
memory usage: 83.6+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Land,Latitude,Longitude
0,Windkraftanlagen Afferde,"2017,",6.600,2,Vestas V126-3.3MW (1×)Nordex N131/3300 (1×),Afferde,HM,"52° 6′ 11″ N, 9° 25′ 28″ O",Landwind,,2017,"HB,HH, H",52° 6′ 11″ N,9° 25′ 28″ O
1,Windkraftanlagen Ahlshausen,"1995,2011,",1.400,2,Tacke TW 600 (1×)Enercon E-53 (1×),Ahlshausen,NOM,"51° 47′ 23″ N, 9° 59′ 6″ O",Eichenauer Energie,2011 Repowering (1× Enercon E-53 statt 1× Ener...,1995,"HB,HH, H",51° 47′ 23″ N,9° 59′ 6″ O
2,Windkraftanlage Aldorf,"2016,",3.300,1,Vestas V112-3.3MW (1×),Aldorf,DH,"52° 15′ 6″ N, 8° 29′ 31″ O",,,2016,"HB,HH, H",52° 15′ 6″ N,8° 29′ 31″ O
3,Windkraftanlagen Aldrup,"2004,",2.600,2,Nordex N60/1300 (2×),Aldrup,OL,"52° 49′ 35″ N, 8° 26′ 34″ O",EBV,,2004,"HB,HH, H",52° 49′ 35″ N,8° 26′ 34″ O
4,Windkraftanlage Altenesch,"2014,",3.050,1,Enercon E-101 (1×),Altenesch,BRA,"53° 6′ 29″ N, 8° 37′ 28″ O",,,2014,"HB,HH, H",53° 6′ 29″ N,8° 37′ 28″ O
5,Windkraftanlagen Altenwerder,"2008,–200,9",12.000,2,Enercon E-126 (2×),Altenwerder,HH,"53° 30′ 34″ N, 9° 55′ 3″ O",Windpark GmbH,Errichtet an der A 7,2008,"HB,HH, H",53° 30′ 34″ N,9° 55′ 3″ O
6,Windkraftanlage Andorf,"1994,",0.080,1,Tacke TW 80 (1×),Andorf,OS,"52° 39′ 50″ N, 7° 48′ 54″ O",,,1994,"HB,HH, H",52° 39′ 50″ N,7° 48′ 54″ O
7,Windkraftanlage Apeldorn,"1995,",0.500,1,Enercon E-40/5.40 (1×),Apeldorn,EL,"52° 44′ 57″ N, 7° 24′ 43″ O",,,1995,"HB,HH, H",52° 44′ 57″ N,7° 24′ 43″ O
8,Windkraftanlage Ardorf,"1991,1997,",0.225,1,Vestas V27-225kW (1×),Ardorf,WTM,"53° 31′ 40″ N, 7° 42′ 58″ O",,Eine Enercon E-30/2.30 wurde 2022 abgebaut,1991,"HB,HH, H",53° 31′ 40″ N,7° 42′ 58″ O
9,Windkraftanlage Aschhorn,"1992,",0.225,1,Vestas V27-225kW (1×),Aschhorn,STD,"53° 41′ 35″ N, 9° 20′ 33″ O",Morgenwind GbR,,1992,"HB,HH, H",53° 41′ 35″ N,9° 20′ 33″ O


In [363]:
orts_nord=df_nord.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_nord

,Anzahl
Landkreis,
EL,512
CUX,511
AUR,495
DH,406
CLP,233
...,...
DEL,5
WOB,5
BS,5


In [364]:

orts_nord_MW=df_nord.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_nord_MW

,Gesamtleistung (MW)
Landkreis,
EL,1338.630
DH,941.555
AUR,873.855
CUX,797.325
STD,574.375
...,...
DEL,6.000
HSHG,5.000
WOB,3.400


In [470]:
hbh=df_nord[['Name', 'Baujahr','Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
hbh.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlagen Afferde,"2017,",2017,6.6,2,Vestas V126-3.3MW (1×)Nordex N131/3300 (1×),Afferde,HM,Landwind,,52° 6′ 11″ N,9° 25′ 28″ O,"HB,HH, H"
1,Windkraftanlagen Ahlshausen,"1995,2011,",1995,1.4,2,Tacke TW 600 (1×)Enercon E-53 (1×),Ahlshausen,NOM,Eichenauer Energie,2011 Repowering (1× Enercon E-53 statt 1× Ener...,51° 47′ 23″ N,9° 59′ 6″ O,"HB,HH, H"
2,Windkraftanlage Aldorf,"2016,",2016,3.3,1,Vestas V112-3.3MW (1×),Aldorf,DH,,,52° 15′ 6″ N,8° 29′ 31″ O,"HB,HH, H"


In [471]:
print(hbh['Landkreis'].unique())

dict_hbh={'ROWSTD':'ROW',
        'HPE':'H',
        'OSVEC':'OS',
        'PESZHI':'PE',
        'WTMFRI':'WTM',
        'HSHG':'H',
        'OLCLP':'OL',
        'LERWST':'LER',
        'HEMF':'HE',
        'WFSZ':'WF',
        'WTMAUR':'WTM',
        'SHGH':'SHG',
         'HHI':'H',
         'VERROW':'VER',
         'BRAOL':'BRA',
         'SZPE':'SZ',
         'OHZROW':'OHZ',
          'GFWOB':'GF',
         'RI':'SHG',
         'HEWF':'HE',
         #'BH':'HB',
         'GIF':'GF'}

hbh['Landkreis'].replace(dict_hbh, inplace=True)
print(hbh['Landkreis'].unique())

['HM' 'NOM' 'DH' 'OL' 'BRA' 'HH' 'OS' 'EL' 'WTM' 'STD' 'AUR' 'LG' 'EMD'
 'CUX' 'LER' 'HI' 'HB' 'ROW' 'WF' 'NI' 'SHG' 'CLP' 'FRI' 'DEL' 'VEC' 'WL'
 'H' 'HK' 'WHV' 'GÖ' 'HE' 'PE' 'UE' 'BH' 'OHZ' 'CE' 'VER' 'DAN' 'NOH'
 'HOL' 'SZ' 'GF' 'RI' 'GS' 'ROWSTD' 'HPE' 'OSVEC' 'PESZHI' 'WTMFRI' 'GIF'
 'HSHG' 'OLCLP' 'WST' 'LERWST' 'BS' 'HEWF' 'WFSZ' 'WTMAUR' 'SHGH' 'HHI'
 'VERROW' 'BRAOL' 'SZPE' 'WOB' 'OHZROW' 'GFWOB']
['HM' 'NOM' 'DH' 'OL' 'BRA' 'HH' 'OS' 'EL' 'WTM' 'STD' 'AUR' 'LG' 'EMD'
 'CUX' 'LER' 'HI' 'HB' 'ROW' 'WF' 'NI' 'SHG' 'CLP' 'FRI' 'DEL' 'VEC' 'WL'
 'H' 'HK' 'WHV' 'GÖ' 'HE' 'PE' 'UE' 'BH' 'OHZ' 'CE' 'VER' 'DAN' 'NOH'
 'HOL' 'SZ' 'GF' 'GS' 'WST' 'BS' 'WOB']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/916964236.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hbh['Landkreis'].replace(dict_hbh, inplace=True)


### Hessen:

In [368]:
url_he="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Hessen"

# use requests.get() method with the provided static_url

response_he=requests.get(url_he).text
# assign the response to a object
soup_he=BeautifulSoup(response_he, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_he.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_he.find_all('table')
table_he = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_he.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Hessen is:', len(filtered_rows))


column_names = []
columns=table_he.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 
print('We can see that these are the same columns as for BW')



The title is:
Liste von Windkraftanlagen in Hessen – Wikipedia
The number of windparks in Hessen is: 220
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']
We can see that these are the same columns as for BW


In [369]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_he= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })

In [370]:
print(df_he.info())

df_he['Land']='Hessen'

df_he['Anzahl']=df_he['Anzahl'].fillna(0)
# Remove the brackets and their contents
df_he['Anzahl'] = df_he['Anzahl'].str.replace(r'\[\d+\]', '', regex=True)

df_he['Anzahl'] = [int(str(i)) for i in df_he['Anzahl']]
df_he['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_he['Gesamtleistung (MW)']]

fn = lambda x: pd.Series([i for i in (x.split(','))])
pad = df_he['Koordinaten'].apply(fn)

pad.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_he['Latitude']=pad['Latitude']
df_he['Longitude']=pad['Longitude']

df_he.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      219 non-null    object
 1   Baujahr                   219 non-null    object
 2   Gesamtleistung (MW)       219 non-null    object
 3   Anzahl                    219 non-null    object
 4   Typ (WKA)                 219 non-null    object
 5   Ort                       219 non-null    object
 6   Landkreis                 219 non-null    object
 7   Koordinaten               219 non-null    object
 8   Projektierer / Betreiber  219 non-null    object
 9   Bemerkungen               219 non-null    object
 10  Anfangjahr                219 non-null    object
dtypes: object(11)
memory usage: 18.9+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Land,Latitude,Longitude
0,Windkraftanlage Breidenbach (Windpark Hesselbach),"2021,",3.30,1,Vestas V126-3.3MW (1×),Breidenbach,MR,"50° 53′ 38″ N, 8° 23′ 53″ O",Boxbach Energy,zugehörig zum WP Hesselbach auf nordrhein-west...,2021,Hessen,50° 53′ 38″ N,8° 23′ 53″ O
1,Windkraftanlage Friedewald,"2002,",0.60,1,Enercon E-40/6.44 (1×),Friedewald,HEF,"50° 53′ 0″ N, 9° 50′ 10″ O",Windenergie Friedewald,Errichtet an der A 4,2002,Hessen,50° 53′ 0″ N,9° 50′ 10″ O
2,Windkraftanlage Heitzelberg,"2023,",5.50,1,GE Wind Energy 5.5-158 (1×),Freienhagen,KB,"50° 15′ 32″ N, 9° 4′ 2″ O",PNE,,2023,Hessen,50° 15′ 32″ N,9° 4′ 2″ O
3,Windkraftanlage Hombergshausen,"1994,",0.15,1,AN Bonus 150/30 (1×),Hombergshausen,HR,"51° 3′ 47″ N, 9° 26′ 11″ O",L + W Windkraft,,1994,Hessen,51° 3′ 47″ N,9° 26′ 11″ O
4,Windkraftanlage Hünfeld-Michelsrombach,"2024,",4.80,1,Nordex N133/4800 (1×),Michelsrombach,FD,"50° 40′ 36″ N, 9° 40′ 12″ O",ABO Wind,in Bauerrichtet an der A 7Windkraftanlage wird...,2024,Hessen,50° 40′ 36″ N,9° 40′ 12″ O
5,Windkraftanlage Körle,"2002,",1.50,1,REpower MD77 (1×),Körle,HR,"51° 11′ 10″ N, 9° 30′ 32″ O",LuV Windenergie,eine weitere REpower MD77 2020 abgebrannt[6],2002,Hessen,51° 11′ 10″ N,9° 30′ 32″ O
6,Windkraftanlage Lamsberg,"1998,",0.23,1,Enercon E-30/2.30 (1×),Gudensberg,HR,"51° 10′ 23″ N, 9° 23′ 17″ O",Adolf Weingart,,1998,Hessen,51° 10′ 23″ N,9° 23′ 17″ O
7,Windkraftanlage Lindenholzhausen,"1995,",0.50,1,Enercon E-40/5.40 (1×),Lindenholzhausen,LM,"50° 22′ 4″ N, 8° 7′ 11″ O","NET, Kloft",errichtet in Gewerbegebiet an der A 3,1995,Hessen,50° 22′ 4″ N,8° 7′ 11″ O
8,Windkraftanlage Mauswinkel,"1998,",0.60,1,Wind World W4200/600 (1×),Mauswinkel,MKK,"50° 24′ 30″ N, 9° 18′ 26″ O","HessenEnergie, Kreiswerke Main-Kinzig","vorher Windpark, weitgehender Rückbau 2014 (2×...",1998,Hessen,50° 24′ 30″ N,9° 18′ 26″ O
9,Windkraftanlage Münchhausen,"1995,",0.50,1,Enercon E-40/5.40 (1×),Münchhausen,LDK,"50° 36′ 44″ N, 8° 11′ 40″ O",,,1995,Hessen,50° 36′ 44″ N,8° 11′ 40″ O


In [371]:
orts_he=df_he.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_he.head(5)

,Anzahl
Landkreis,
VB,229
KB,172
KS,157
MKK,109
MR,91


In [247]:
orts_he_MW=df_he.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_he_MW.head(5)

,Gesamtleistung (MW)
Landkreis,
VB,457.80
KB,374.06
KS,325.20
MKK,287.95
HEF,253.50


In [372]:
he=df_he[['Name', 'Baujahr','Anfangjahr','Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
he.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Breidenbach (Windpark Hesselbach),"2021,",2021,3.3,1,Vestas V126-3.3MW (1×),Breidenbach,MR,Boxbach Energy,zugehörig zum WP Hesselbach auf nordrhein-west...,50° 53′ 38″ N,8° 23′ 53″ O,Hessen
1,Windkraftanlage Friedewald,"2002,",2002,0.6,1,Enercon E-40/6.44 (1×),Friedewald,HEF,Windenergie Friedewald,Errichtet an der A 4,50° 53′ 0″ N,9° 50′ 10″ O,Hessen
2,Windkraftanlage Heitzelberg,"2023,",2023,5.5,1,GE Wind Energy 5.5-158 (1×),Freienhagen,KB,PNE,,50° 15′ 32″ N,9° 4′ 2″ O,Hessen


In [445]:
print(he['Landkreis'].unique())

dict_he={'LDKLM':'LDK',
        'LDKMR':'LDK',
        'VBMKK':'VB',
        'HEFVB':'HEF',
        'MRGI':'MR', 'LK':'KS'}

he['Landkreis'].replace(dict_he, inplace=True)
print(he['Landkreis'].unique())

['MR' 'HEF' 'KB' 'HR' 'FD' 'LM' 'MKK' 'LDK' 'VB' 'KS' 'LK' 'ESW' 'FB'
 'RÜD' 'DA' 'ERB' 'HG' 'HP' 'GI']
['MR' 'HEF' 'KB' 'HR' 'FD' 'LM' 'MKK' 'LDK' 'VB' 'KS' 'ESW' 'FB' 'RÜD'
 'DA' 'ERB' 'HG' 'HP' 'GI']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/338930294.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  he['Landkreis'].replace(dict_he, inplace=True)


### Mecklenburg-Vorpommen:

In [374]:
url_mp="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Mecklenburg-Vorpommern"
# use requests.get() method with the provided static_url

response_mp=requests.get(url_mp).text
# assign the response to a object
soup_mp=BeautifulSoup(response_mp, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_mp.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_mp.find_all('table')
table_mp = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_mp.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  

print('The number of windparks in Mecklenburg-Vorpommen:', len(filtered_rows))


column_names = []
columns=table_mp.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 


The title is:
Liste von Windkraftanlagen in Mecklenburg-Vorpommern – Wikipedia
The number of windparks in Mecklenburg-Vorpommen: 208
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [375]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_mp= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })


In [376]:
print(df_mp.info())

df_mp['Land']='MP'

df_mp['Anzahl']=df_mp['Anzahl'].fillna(0)
df_mp['Anzahl'] = [int(str(i)) for i in df_mp['Anzahl']]
df_mp['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_mp['Gesamtleistung (MW)']]

fn = lambda x: pd.Series([i for i in (x.split(','))])
pad = df_mp['Koordinaten'].apply(fn)

pad.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_mp['Latitude']=pad['Latitude']
df_mp['Longitude']=pad['Longitude']

df_mp.head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      207 non-null    object
 1   Baujahr                   207 non-null    object
 2   Gesamtleistung (MW)       207 non-null    object
 3   Anzahl                    207 non-null    object
 4   Typ (WKA)                 207 non-null    object
 5   Ort                       207 non-null    object
 6   Landkreis                 207 non-null    object
 7   Koordinaten               207 non-null    object
 8   Projektierer / Betreiber  207 non-null    object
 9   Bemerkungen               207 non-null    object
 10  Anfangjahr                207 non-null    object
dtypes: object(11)
memory usage: 17.9+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Land,Latitude,Longitude
0,Windkraftanlage Altenkirchen,"1993,",0.500,1,Enercon E-40/5.40 (1×),Altenkirchen,VR,"54° 37′ 42″ N, 13° 19′ 55″ O",,,1993,MP,54° 37′ 42″ N,13° 19′ 55″ O
1,Windkraftanlage Borg,"2007,",2.000,1,Enercon E-70 (1×),Ribnitz-Damgarten,VR,"54° 14′ 11″ N, 12° 23′ 15″ O",WIND-Projekt,früher noch 1× Lagerwey LW15/75,2007,MP,54° 14′ 11″ N,12° 23′ 15″ O
2,Windkraftanlage Breitling,"2005,",2.500,1,Nordex N90/2500 (1×),Peez,HRO,"54° 9′ 25″ N, 12° 7′ 33″ O","WIND-projekt, Nordex",Testanlage zur Simulation eines Standortes unt...,2005,MP,54° 9′ 25″ N,12° 7′ 33″ O
3,Windkraftanlage Dalkendorf,"1999,",0.660,1,Vestas V47-660kW (1×),Dalkendorf,LRO,"53° 49′ 11″ N, 12° 30′ 1″ O",,,1999,MP,53° 49′ 11″ N,12° 30′ 1″ O
4,Windkraftanlage Dargast,"2000,",0.300,1,AN Bonus 300/33 (1×),Sassnitz,VR,"54° 31′ 46″ N, 13° 36′ 38″ O",,,2000,MP,54° 31′ 46″ N,13° 36′ 38″ O
5,Windkraftanlage Demnitz,"1996,",0.600,1,Tacke TW 600 (1×),Altwigshagen,VG,"53° 41′ 41″ N, 13° 48′ 53″ O",,,1996,MP,53° 41′ 41″ N,13° 48′ 53″ O
6,Windkraftanlage Ducherow-Altwigshagen,"2022,",6.000,1,Vestas V150-6.0MW (1×),Altwigshagen,VG,"53° 43′ 58″ N, 13° 48′ 30″ O",Notus Energy,,2022,MP,53° 43′ 58″ N,13° 48′ 30″ O
7,Windkraftanlage Dumsevitz,"2002,",0.600,1,Enercon E-40/6.44 (1×),Garz,VR,"54° 17′ 34″ N, 13° 21′ 31″ O",,,2002,MP,54° 17′ 34″ N,13° 21′ 31″ O
8,Windkraftanlage Fahrbinde,"1997,",0.500,1,Enercon E-40/5.40 (1×),Fahrbinde,LUP,"53° 26′ 28″ N, 11° 29′ 31″ O",Landwirtschaftl. Betrieb,Errichtet an der A 14,1997,MP,53° 26′ 28″ N,11° 29′ 31″ O
9,Windkraftanlagen Franzburg,"2023,",12.000,2,eno energy eno160-6.0MW (2×),Franzburg,VR,"54° 11′ 37″ N, 12° 50′ 17″ O",eno energy,in BauPrototypen der eno energy eno160weitere ...,2023,MP,54° 11′ 37″ N,12° 50′ 17″ O


In [448]:
#we have to do it because in wikipedia park for this windpark
#the coordinates are absent
df_mp['Landkreis'].loc[df_mp['Name'].str.contains('Windpark Stretense')]= str('VG')


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/4292567437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mp['Landkreis'].loc[df_mp['Name'].str.contains('Windpark Stretense')]= str('VG')


In [466]:
orts_mp_MW=df_mp.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_mp_MW

,Gesamtleistung (MW)
Landkreis,
LUP,826.080
LRO,803.930
VG,714.400
MSE,705.260
VR,455.905
NWM,344.455
VGVR,61.570
HRO,8.400


In [467]:
orts_mp=df_mp.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_mp

,Anzahl
Landkreis,
VG,391
LUP,359
LRO,344
MSE,317
VR,273
NWM,173
VGVR,26
HRO,9


In [468]:
mvp=df_mp[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
mvp.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Altenkirchen,"1993,",1993,0.5,1,Enercon E-40/5.40 (1×),Altenkirchen,VR,,,54° 37′ 42″ N,13° 19′ 55″ O,MP
1,Windkraftanlage Borg,"2007,",2007,2.0,1,Enercon E-70 (1×),Ribnitz-Damgarten,VR,WIND-Projekt,früher noch 1× Lagerwey LW15/75,54° 14′ 11″ N,12° 23′ 15″ O,MP
2,Windkraftanlage Breitling,"2005,",2005,2.5,1,Nordex N90/2500 (1×),Peez,HRO,"WIND-projekt, Nordex",Testanlage zur Simulation eines Standortes unt...,54° 9′ 25″ N,12° 7′ 33″ O,MP


In [469]:
print(mvp['Landkreis'].unique())

dict_mvp={'VGVR':'VG'}
         

mvp['Landkreis'].replace(dict_mvp, inplace=True)
print(mvp['Landkreis'].unique())

['VR' 'HRO' 'LRO' 'VG' 'LUP' 'NWM' 'MSE' 'VGVR']
['VR' 'HRO' 'LRO' 'VG' 'LUP' 'NWM' 'MSE']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/1024598736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvp['Landkreis'].replace(dict_mvp, inplace=True)


### Nordrhein-Westfalien:

In [381]:
url_nw="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Nordrhein-Westfalen"
# use requests.get() method with the provided static_url
response_nw=requests.get(url_nw).text
# assign the response to a object
soup_nw=BeautifulSoup(response_nw, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_nw.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_nw.find_all('table')
table_nw = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_nw.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Nordrhein-Westphalien is:', len(filtered_rows))


column_names = []
columns=table_be_ba.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 


The title is:
Liste von Windkraftanlagen in Nordrhein-Westfalen – Wikipedia
The number of windparks in Nordrhein-Westphalien is: 804
['Name', 'Baujahr', 'Gesamt-  Leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land-  kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [382]:
 # Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_nw= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })
                             
  
    

In [383]:
print(df_nw.info())

#splitting the coordinates:
fn = lambda x: pd.Series([i for i in (x.split('N,'))])
coord = df_nw['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_nw['Latitude']=coord['Latitude']
df_nw['Longitude']=coord['Longitude']


df_nw['Land']='NW'
#df_nw['Anzahl']=df_nw['Anzahl'].fillna(0)
df_nw['Anzahl'] = [int(str(i)) for i in df_nw['Anzahl']]
df_nw['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_nw['Gesamtleistung (MW)']]


df_nw.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803 entries, 0 to 802
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      803 non-null    object
 1   Baujahr                   803 non-null    object
 2   Gesamtleistung (MW)       803 non-null    object
 3   Anzahl                    803 non-null    object
 4   Typ (WKA)                 803 non-null    object
 5   Ort                       803 non-null    object
 6   Landkreis                 803 non-null    object
 7   Koordinaten               803 non-null    object
 8   Projektierer / Betreiber  803 non-null    object
 9   Bemerkungen               803 non-null    object
 10  Anfangjahr                803 non-null    object
dtypes: object(11)
memory usage: 69.1+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Windkraftanlage Affeln,"1994,",0.6,1,Tacke TW 600 (1×),Affeln,MK,"51,27842° N, 7,89391° O",Windinvestor,,1994,"51,27842°","7,89391° O",NW
1,Windkraftanlagen Ahlen-Gemmerich,"2023,",8.4,2,Enercon E-138 EP3 E2 (2×),Gemmerich,WAF,"51,73053° N, 7,884509° O",Vento Ludens GmbH ＆Co.KG,,2023,"51,73053°","7,884509° O",NW
2,Windkraftanlagen Ahlen-Schäringerfeld,"2020,-202,1",9.0,2,Nordex N149/4500 (2×),Vorhelm,WAF,"51,814662° N, 7,960509° O",Drehstrom AG,,2020,"51,814662°","7,960509° O",NW


In [384]:
#Coordinates for NRW are in decimals 
df_nw['Latitude']=df_nw['Latitude'].apply(lambda x: x.replace(',', '.'))
df_nw['Longitude']=df_nw['Longitude'].apply(lambda x: x.replace(',', '.'))

In [385]:
orts_nw_MW=df_nw.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_nw_MW.head(10)

,Gesamtleistung (MW)
Landkreis,
PB,1409.985
ST,699.405
BOR,669.880
DN,533.140
WAF,458.130
SO,427.720
HSK,415.875
KLE,397.620
HX,373.470


In [386]:
orts_nw=df_nw.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_nw.head(10)

,Anzahl
Landkreis,
PB,548
ST,299
BOR,284
SO,265
WAF,212
KLE,196
HX,186
DN,174
LIP,152


In [387]:
nw=df_nw[['Name', 'Baujahr', 'Anfangjahr','Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
nw.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Affeln,"1994,",1994,0.6,1,Tacke TW 600 (1×),Affeln,MK,Windinvestor,,51.27842°,7.89391° O,NW
1,Windkraftanlagen Ahlen-Gemmerich,"2023,",2023,8.4,2,Enercon E-138 EP3 E2 (2×),Gemmerich,WAF,Vento Ludens GmbH ＆Co.KG,,51.73053°,7.884509° O,NW
2,Windkraftanlagen Ahlen-Schäringerfeld,"2020,-202,1",2020,9.0,2,Nordex N149/4500 (2×),Vorhelm,WAF,Drehstrom AG,,51.814662°,7.960509° O,NW


In [388]:
print(nw['Landkreis'].unique())

dict_nw={'HAMK':'HA',
        'OESI':'OE',
        'ACDN':'AC',
        'HSDN':'HS',
         'HSKPB':'HS',
        'REWES':'RE',
        'BMNE':'BM',
        'WESRE':'WES',
        'BORST':'BOR',
        'MIHF':'MI',
        'BIGT':'BI',
        'OWL':'BI', 'BMDN':'BM'}

nw['Landkreis'].replace(dict_nw, inplace=True)
print(nw['Landkreis'].unique())

['MK' 'WAF' 'AC' 'HX' 'LIP' 'UN' 'COE' 'OE' 'KLE' 'GT' 'RE' 'DN' 'EN' 'GM'
 'EU' 'HF' 'SO' 'MI' 'BOR' 'BOT' 'VIE' 'PB' 'WES' 'BM' 'ST' 'MS' 'HSK'
 'DO' 'HS' 'SI' 'HA' 'GE' 'HAM' 'ME' 'BI' 'NE' 'RS' 'MH' 'GL' 'W' 'HAMK'
 'OESI' 'BMDN' 'ACDN' 'BIGT' 'KR' 'HSDN' 'HSKPB' 'MG' 'REWES' 'BMNE'
 'WESRE' 'BORST' 'SU' 'MIHF']
['MK' 'WAF' 'AC' 'HX' 'LIP' 'UN' 'COE' 'OE' 'KLE' 'GT' 'RE' 'DN' 'EN' 'GM'
 'EU' 'HF' 'SO' 'MI' 'BOR' 'BOT' 'VIE' 'PB' 'WES' 'BM' 'ST' 'MS' 'HSK'
 'DO' 'HS' 'SI' 'HA' 'GE' 'HAM' 'ME' 'BI' 'NE' 'RS' 'MH' 'GL' 'W' 'KR'
 'MG' 'SU']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/3967266508.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nw['Landkreis'].replace(dict_nw, inplace=True)


### Rheinland-Pfalz:

In [389]:
url_rp="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Rheinland-Pfalz"

response_rp=requests.get(url_rp).text
# assign the response to a object
soup_rp=BeautifulSoup(response_rp, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_rp.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_rp.find_all('table')
table_rp = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_rp.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Rheinland/Pfalz is:', len(filtered_rows))


column_names = []
columns=table_rp.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 



The title is:
Liste von Windkraftanlagen in Rheinland-Pfalz – Wikipedia
The number of windparks in Rheinland/Pfalz is: 300
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [390]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_rp= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })


In [391]:
print(df_rp.info())
#splitting the coordinates:
fn = lambda x: pd.Series([i for i in (x.split(','))])
coord = df_rp['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_rp['Latitude']=coord['Latitude']
df_rp['Longitude']=coord['Longitude']


df_rp['Land']='RP'

df_rp['Anzahl'] = [int(str(i)) for i in df_rp['Anzahl']]
df_rp['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_rp['Gesamtleistung (MW)']]


df_rp.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      299 non-null    object
 1   Baujahr                   299 non-null    object
 2   Gesamtleistung (MW)       299 non-null    object
 3   Anzahl                    299 non-null    object
 4   Typ (WKA)                 299 non-null    object
 5   Ort                       299 non-null    object
 6   Landkreis                 299 non-null    object
 7   Koordinaten               299 non-null    object
 8   Projektierer / Betreiber  299 non-null    object
 9   Bemerkungen               299 non-null    object
 10  Anfangjahr                299 non-null    object
dtypes: object(11)
memory usage: 25.8+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Windkraftanlage Badenheim,"2016,",3.05,1,Enercon E-101 (1×),Badenheim,MZ,"49° 49′ 58″ N, 7° 59′ 15″ O",GAIA,,2016,49° 49′ 58″ N,7° 59′ 15″ O,RP
1,Windkraftanlage Berghausen,"1996,2001,",1.30,1,AN Bonus 1300/62 (1×),Berghausen,EMS,"50° 15′ 24″ N, 8° 1′ 0″ O",VS Windkraft,eine Seewind 25/132 wurde 2013 zugunsten eines...,1996,50° 15′ 24″ N,8° 1′ 0″ O,RP
2,Windkraftanlage Burg,"2001,",1.50,1,Südwind S70 (1×),Burg,BIT,"49° 57′ 8″ N, 6° 21′ 0″ O",,,2001,49° 57′ 8″ N,6° 21′ 0″ O,RP


In [392]:
orts_rp_MW=df_rp.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_rp_MW.head(10)

,Gesamtleistung (MW)
Landkreis,
SIM,676.45
BIT,465.92
AZ,411.98
TR,242.60
WIL,241.87
KIB,220.60
DAU,186.65
BIR,167.40
WW,165.38


In [393]:
orts_rp=df_rp.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_rp.head(10)

,Anzahl
Landkreis,
SIM,268
BIT,259
AZ,135
DAU,111
WW,100
TR,100
COC,86
WIL,71
KIB,70


In [394]:
rp=df_rp[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
rp.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Badenheim,"2016,",2016,3.05,1,Enercon E-101 (1×),Badenheim,MZ,GAIA,,49° 49′ 58″ N,7° 59′ 15″ O,RP
1,Windkraftanlage Berghausen,"1996,2001,",1996,1.30,1,AN Bonus 1300/62 (1×),Berghausen,EMS,VS Windkraft,eine Seewind 25/132 wurde 2013 zugunsten eines...,50° 15′ 24″ N,8° 1′ 0″ O,RP
2,Windkraftanlage Burg,"2001,",2001,1.50,1,Südwind S70 (1×),Burg,BIT,,,49° 57′ 8″ N,6° 21′ 0″ O,RP


In [395]:
print(rp['Landkreis'].unique())

dict_rp={'DÜWRP':'DÜW',
        'AZMZ':'AZ',
        'MZMZ':'MZ',
        'BIRKUS':'BIR',
         'AZKIB':'AZ',
        'KHKIB':'KH',
        'KLPS':'KL',
        'AZDÜWKIB':'AZ',
        'KHSIMMZ':'KH',
        'COCSIM':'COC',
        'KUSKIB':'KUS',
        'RHKMZ':'SIM',
        'GERSÜW':'GER',
        'SIMKH':'SIM',
        'AWMYK':'AW',
        'TRBIT':'TR',
        'AZWO':'AZ',
        'KUSKL':'KUS',
        'KLKUS':'KL',
        'MAY':'MYK',
        'RHK':'SIM'}

rp['Landkreis'].replace(dict_rp, inplace=True)
print(rp['Landkreis'].unique())


['MZ' 'EMS' 'BIT' 'BIR' 'KL' 'WW' 'KIB' 'WIL' 'PS' 'SIM' 'DAU' 'AK' 'COC'
 'AZ' 'RP' 'KH' 'KUS' 'DÜW' 'MYK' 'DÜWRP' 'AZMZ' 'MZMZ' 'BIRKUS' 'AZKIB'
 'GER' 'KHKIB' 'TR' 'KLPS' 'KLKUS' 'AZDÜWKIB' 'KHSIMMZ' 'COCSIM' 'MAY'
 'KUSKIB' 'SÜW' 'RHKMZ' 'KUSKL' 'GERSÜW' 'SIMKH' 'AWMYK' 'TRBIT' 'AZWO']
['MZ' 'EMS' 'BIT' 'BIR' 'KL' 'WW' 'KIB' 'WIL' 'PS' 'SIM' 'DAU' 'AK' 'COC'
 'AZ' 'RP' 'KH' 'KUS' 'DÜW' 'MYK' 'GER' 'TR' 'SÜW' 'AW']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/3673199108.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rp['Landkreis'].replace(dict_rp, inplace=True)


### Saarland:

In [396]:
url_sa="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_im_Saarland"
# use requests.get() method with the provided static_url
response_sa=requests.get(url_sa).text
# assign the response to a object
soup_sa=BeautifulSoup(response_sa, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_sa.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_sa.find_all('table')
table_sa = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_sa.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Saarland is:', len(filtered_rows))


column_names = []
columns=table_be_ba.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 



The title is:
Liste von Windkraftanlagen im Saarland – Wikipedia
The number of windparks in Saarland is: 51
['Name', 'Baujahr', 'Gesamt-  Leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land-  kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [397]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_sa= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })

    

In [398]:
print(df_sa.info())
#splitting the coordinates:
fn = lambda x: pd.Series([i for i in (x.split(','))])
coord = df_sa['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_sa['Latitude']=coord['Latitude']
df_sa['Longitude']=coord['Longitude']

df_sa['Land']='SA'
#df_nw['Anzahl']=df_nw['Anzahl'].fillna(0)
df_sa['Anzahl'] = [int(str(i)) for i in df_sa['Anzahl']]
df_sa['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_sa['Gesamtleistung (MW)']]


df_sa.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      50 non-null     object
 1   Baujahr                   50 non-null     object
 2   Gesamtleistung (MW)       50 non-null     object
 3   Anzahl                    50 non-null     object
 4   Typ (WKA)                 50 non-null     object
 5   Ort                       50 non-null     object
 6   Landkreis                 50 non-null     object
 7   Koordinaten               50 non-null     object
 8   Projektierer / Betreiber  50 non-null     object
 9   Bemerkungen               50 non-null     object
 10  Anfangjahr                50 non-null     object
dtypes: object(11)
memory usage: 4.4+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Windkraftanlage Erkershöhe,"2018,",3.00,1,Nordex N131/3000 (1×),Wemmetsweiler,NK,"49° 20′ 31″ N, 7° 4′ 30″ O",montanWIND GmbH,,2018,49° 20′ 31″ N,7° 4′ 30″ O,SA
1,Windkraftanlage Riesweiler,"1999,",0.25,1,Nordex N29/250 (1×),Riesweiler,HOM,"49° 8′ 3″ N, 7° 18′ 22″ O",,,1999,49° 8′ 3″ N,7° 18′ 22″ O,SA
2,Windkraftanlagen Gersweiler,"2022,-202,3",9.76,2,Enercon E-138 EP3 E2 (1×)Enercon E-160 EP5 E3 ...,Gersweiler,SB,"49° 12′ 35″ N, 6° 53′ 32″ O",DunoAir,,2022,49° 12′ 35″ N,6° 53′ 32″ O,SA


In [399]:
orts_sa_MW=df_sa.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_sa_MW.head(10)

,Gesamtleistung (MW)
Landkreis,
MZG,177.70
WND,168.05
SLS,63.50
NKWND,51.35
NK,35.50
WNDMZG,24.05
NKHOM,16.50
HOM,16.00
SB,15.76


In [400]:
orts_sa=df_sa.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_sa.head(10)

,Anzahl
Landkreis,
WND,71
MZG,68
SLS,21
NKWND,19
NK,16
WNDMZG,10
HOM,8
NKHOM,5
SB,4


In [401]:
sa=df_sa[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
sa.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Erkershöhe,"2018,",2018,3.00,1,Nordex N131/3000 (1×),Wemmetsweiler,NK,montanWIND GmbH,,49° 20′ 31″ N,7° 4′ 30″ O,SA
1,Windkraftanlage Riesweiler,"1999,",1999,0.25,1,Nordex N29/250 (1×),Riesweiler,HOM,,,49° 8′ 3″ N,7° 18′ 22″ O,SA
2,Windkraftanlagen Gersweiler,"2022,-202,3",2022,9.76,2,Enercon E-138 EP3 E2 (1×)Enercon E-160 EP5 E3 ...,Gersweiler,SB,DunoAir,,49° 12′ 35″ N,6° 53′ 32″ O,SA


In [402]:
print(sa['Landkreis'].unique())

dict_sa={
        'NKWND':'NK',
        'NKHOM':'NK',
        'WNDMZG':'WND'}

sa['Landkreis'].replace(dict_sa, inplace=True)
print(sa['Landkreis'].unique())


['NK' 'HOM' 'SB' 'SLS' 'MZG' 'WND' 'NKWND' 'NKHOM' 'WNDMZG']
['NK' 'HOM' 'SB' 'SLS' 'MZG' 'WND']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/1091307559.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa['Landkreis'].replace(dict_sa, inplace=True)


### Sachsen:

In [403]:
url_s="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Sachsen"
# use requests.get() method with the provided static_url
response_s=requests.get(url_s).text
# assign the response to a object
soup_s=BeautifulSoup(response_s, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_s.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_s.find_all('table')
table_s = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_s.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Sachsen is:', len(filtered_rows))


column_names = []
columns=table_s.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 




The title is:
Liste von Windkraftanlagen in Sachsen – Wikipedia
The number of windparks in Sachsen is: 171
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [404]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_s= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })

In [405]:
print(df_s.info())

#splitting the coordinates:
fn = lambda x: pd.Series([i for i in (x.split(','))])
coord = df_s['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_s['Latitude']=coord['Latitude']
df_s['Longitude']=coord['Longitude']


df_s['Land']='SA'
#df_nw['Anzahl']=df_nw['Anzahl'].fillna(0)
df_s['Anzahl'] = [int(str(i)) for i in df_s['Anzahl']]
df_s['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_s['Gesamtleistung (MW)']]


df_s.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      170 non-null    object
 1   Baujahr                   170 non-null    object
 2   Gesamtleistung (MW)       170 non-null    object
 3   Anzahl                    170 non-null    object
 4   Typ (WKA)                 170 non-null    object
 5   Ort                       170 non-null    object
 6   Landkreis                 170 non-null    object
 7   Koordinaten               170 non-null    object
 8   Projektierer / Betreiber  170 non-null    object
 9   Bemerkungen               170 non-null    object
 10  Anfangjahr                170 non-null    object
dtypes: object(11)
memory usage: 14.7+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Windkraftanlage Altenhain,"1995,1999,",0.25,1,Nordex N29/250 (1×),Altenhain,L,"51° 17′ 4″ N, 12° 42′ 16″ O",,eine Nordex N27/150 wurde 2015 abgebaut,1995,51° 17′ 4″ N,12° 42′ 16″ O,SA
1,Windkraftanlage Börnichen,"2002,",1.80,1,Enercon E-66/18.70 (1×),Börnichen,ERZ,"50° 45′ 26″ N, 13° 7′ 40″ O",,,2002,50° 45′ 26″ N,13° 7′ 40″ O,SA
2,Windkraftanlage Ebersdorf,"2004,",0.60,1,Enercon E-40/6.44 (1×),Ebersdorf,GR,"51° 4′ 8″ N, 14° 40′ 39″ O",,,2004,51° 4′ 8″ N,14° 40′ 39″ O,SA


In [406]:
orts_s_MW=df_s.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_s_MW.head(10)

,Gesamtleistung (MW)
Landkreis,
FG,253.530
L,219.810
GR,208.700
MEI,190.770
Z,169.145
BZ,138.210
TDO,107.660
FGERZ,59.000
PIR,46.770


In [407]:
orts_s=df_s.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_s.head(10)

,Anzahl
Landkreis,
FG,172
GR,120
L,99
TDO,90
BZ,86
MEI,79
Z,58
PIR,51
ERZ,40


In [410]:
s=df_s[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
s.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Altenhain,"1995,1999,",1995,0.25,1,Nordex N29/250 (1×),Altenhain,L,,eine Nordex N27/150 wurde 2015 abgebaut,51° 17′ 4″ N,12° 42′ 16″ O,SA
1,Windkraftanlage Börnichen,"2002,",2002,1.80,1,Enercon E-66/18.70 (1×),Börnichen,ERZ,,,50° 45′ 26″ N,13° 7′ 40″ O,SA
2,Windkraftanlage Ebersdorf,"2004,",2004,0.60,1,Enercon E-40/6.44 (1×),Ebersdorf,GR,,,51° 4′ 8″ N,14° 40′ 39″ O,SA


In [411]:
print(s['Landkreis'].unique())

dict_s={
        'TDOL':'TDO',
        'FGERZ':'FG',
        'PIRBZ':'PIR',
          'FGZ':'FG'}

s['Landkreis'].replace(dict_s, inplace=True)
print(s['Landkreis'].unique())


['L' 'ERZ' 'GR' 'FG' 'TDO' 'BZ' 'PIR' 'Z' 'V' 'C' 'MEI' 'TDOL' 'FGERZ'
 'FGZ' 'PIRBZ']
['L' 'ERZ' 'GR' 'FG' 'TDO' 'BZ' 'PIR' 'Z' 'V' 'C' 'MEI']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/3738046354.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['Landkreis'].replace(dict_s, inplace=True)


### Sachsen-Anhalt:

In [412]:
url_an="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Sachsen-Anhalt"
# use requests.get() method with the provided static_url
response_an=requests.get(url_an).text
# assign the response to a object
soup_an=BeautifulSoup(response_an, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_an.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_an.find_all('table')
table_an = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_an.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Sachsen-Anhalt is:', len(filtered_rows))


column_names = []
columns=table_an.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 



The title is:
Liste von Windkraftanlagen in Sachsen-Anhalt – Wikipedia
The number of windparks in Sachsen-Anhalt is: 222
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [413]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_an= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })


In [414]:
print(df_an.info())

#splitting the coordinates:
fn = lambda x: pd.Series([i for i in (x.split(','))])
coord = df_an['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_an['Latitude']=coord['Latitude']
df_an['Longitude']=coord['Longitude']


df_an['Land']='SA'

df_an['Anzahl'] = [int(str(i)) for i in df_an['Anzahl']]
df_an['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_an['Gesamtleistung (MW)']]


df_an.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      221 non-null    object
 1   Baujahr                   221 non-null    object
 2   Gesamtleistung (MW)       221 non-null    object
 3   Anzahl                    221 non-null    object
 4   Typ (WKA)                 221 non-null    object
 5   Ort                       221 non-null    object
 6   Landkreis                 221 non-null    object
 7   Koordinaten               221 non-null    object
 8   Projektierer / Betreiber  221 non-null    object
 9   Bemerkungen               221 non-null    object
 10  Anfangjahr                221 non-null    object
dtypes: object(11)
memory usage: 19.1+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Windkraftanlage Ackendorf,"2003,",1.5,1,GE Wind Energy 1.5sl (1×),Haldensleben,BK,"52° 13′ 29″ N, 11° 25′ 34″ O",Germania Windpark,,2003,52° 13′ 29″ N,11° 25′ 34″ O,SA
1,Windkraftanlage Breitenstein,"1997,",0.5,1,Enercon E-40/5.40 (1×),Breitenstein,MSH,"51° 37′ 26″ N, 10° 56′ 29″ O",,,1997,51° 37′ 26″ N,10° 56′ 29″ O,SA
2,Windkraftanlage Bretsch,"2001,",0.5,1,Enercon E-40/5.40 (1×),Bretsch,SDL,"52° 50′ 13″ N, 11° 38′ 20″ O",,,2001,52° 50′ 13″ N,11° 38′ 20″ O,SA


In [415]:
orts_an=df_an.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_an.head(10)

,Anzahl
Landkreis,
BK,415
SLK,390
SDL,308
BLK,250
ABI,242
SAW,213
MSH,178
SK,171
WB,146


In [416]:
orts_an_MW=df_an.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_an_MW.head(10)

,Gesamtleistung (MW)
Landkreis,
SDL,854.11
BK,725.60
SLK,723.25
BLK,624.94
ABI,505.38
SAW,401.25
WB,369.02
SK,326.55
MSH,287.57


In [417]:
an=df_an[['Name', 'Baujahr','Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
an.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Ackendorf,"2003,",2003,1.5,1,GE Wind Energy 1.5sl (1×),Haldensleben,BK,Germania Windpark,,52° 13′ 29″ N,11° 25′ 34″ O,SA
1,Windkraftanlage Breitenstein,"1997,",1997,0.5,1,Enercon E-40/5.40 (1×),Breitenstein,MSH,,,51° 37′ 26″ N,10° 56′ 29″ O,SA
2,Windkraftanlage Bretsch,"2001,",2001,0.5,1,Enercon E-40/5.40 (1×),Bretsch,SDL,,,52° 50′ 13″ N,11° 38′ 20″ O,SA


In [418]:
print(an['Landkreis'].unique())

dict_an={
        'SKMSH':'SK',
        'SKSLK':'SK',
        'MSHSK':'MSH',
        'JLSDL':'JL',
        'SAWSDL':'SAW',
         'MSHSLK':'MSH',
          'BKHZ':'BK'}

an['Landkreis'].replace(dict_an, inplace=True)
print(an['Landkreis'].unique())

['BK' 'MSH' 'SDL' 'WB' 'HZ' 'SK' 'JL' 'BLK' 'ABI' 'MD' 'DE' 'SAW' 'SLK'
 'SKMSH' 'SKSLK' 'MSHSK' 'JLSDL' 'SAWSDL' 'MSHSLK' 'BKHZ']
['BK' 'MSH' 'SDL' 'WB' 'HZ' 'SK' 'JL' 'BLK' 'ABI' 'MD' 'DE' 'SAW' 'SLK']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/2332430607.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  an['Landkreis'].replace(dict_an, inplace=True)


### Schleswig-Holstein:


In [419]:
url_ho="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Schleswig-Holstein"
# use requests.get() method with the provided static_url
response_ho=requests.get(url_ho).text
# assign the response to a object
soup_ho=BeautifulSoup(response_ho, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_ho.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_ho.find_all('table')
table_ho = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_ho.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Schleswig/Holstein is:', len(filtered_rows))


column_names = []
columns=table_ho.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 


The title is:
Liste von Windkraftanlagen in Schleswig-Holstein – Wikipedia
The number of windparks in Schleswig/Holstein is: 399
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [420]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_ho= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })


In [421]:
print(df_ho.info())


#splitting the coordinates:
fn = lambda x: pd.Series([i for i in (x.split(','))])
coord = df_ho['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_ho['Latitude']=coord['Latitude']
df_ho['Longitude']=coord['Longitude']

df_ho['Land']='Sch-H'

df_ho['Anzahl'] = [int(str(i)) for i in df_ho['Anzahl']]
df_ho['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_ho['Gesamtleistung (MW)']]


df_ho.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      398 non-null    object
 1   Baujahr                   398 non-null    object
 2   Gesamtleistung (MW)       398 non-null    object
 3   Anzahl                    398 non-null    object
 4   Typ (WKA)                 398 non-null    object
 5   Ort                       398 non-null    object
 6   Landkreis                 398 non-null    object
 7   Koordinaten               398 non-null    object
 8   Projektierer / Betreiber  398 non-null    object
 9   Bemerkungen               398 non-null    object
 10  Anfangjahr                398 non-null    object
dtypes: object(11)
memory usage: 34.3+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Windkraftanlage Ahrenshöft,"1998,2009,",1.50,1,Vensys 77/1500 (1×),Ahrenshöft,NF,"54° 33′ 39″ N, 9° 5′ 16″ O",Ahrenshöfter Naturkraftwerke,Von 1998 bis 2003 stand im Windpark eine weite...,1998,54° 33′ 39″ N,9° 5′ 16″ O,Sch-H
1,Windkraftanlage Albersdorf Süd,"2022,",5.56,1,Enercon E-160 EP5 E3 (1×),Albersdorf,HEI,"54° 7′ 18″ N, 9° 15′ 39″ O",WP Schlaa,errichtet an der A 23,2022,54° 7′ 18″ N,9° 15′ 39″ O,Sch-H
2,Windkraftanlagen Arup,"1995,",1.00,2,Nordtank NTK 500/41 (2×),Struxdorf,SL,"54° 37′ 20″ N, 9° 38′ 3″ O",Westangler Windenergie,,1995,54° 37′ 20″ N,9° 38′ 3″ O,Sch-H


In [422]:
orts_ho=df_ho.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_ho.head(10)

,Anzahl
Landkreis,
NF,861
HEI,852
SL,510
OH,337
IZ,296
RD,230
SE,73
RZ,62
OD,41


In [423]:
orts_ho_MW=df_ho.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_ho_MW.head(10)

,Gesamtleistung (MW)
Landkreis,
NF,2508.940
HEI,2361.500
SL,1440.400
IZ,828.000
OH,825.175
RD,693.355
SE,284.180
RZ,104.050
OHSE,91.500


In [424]:
ho=df_ho[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
ho.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Ahrenshöft,"1998,2009,",1998,1.50,1,Vensys 77/1500 (1×),Ahrenshöft,NF,Ahrenshöfter Naturkraftwerke,Von 1998 bis 2003 stand im Windpark eine weite...,54° 33′ 39″ N,9° 5′ 16″ O,Sch-H
1,Windkraftanlage Albersdorf Süd,"2022,",2022,5.56,1,Enercon E-160 EP5 E3 (1×),Albersdorf,HEI,WP Schlaa,errichtet an der A 23,54° 7′ 18″ N,9° 15′ 39″ O,Sch-H
2,Windkraftanlagen Arup,"1995,",1995,1.00,2,Nordtank NTK 500/41 (2×),Struxdorf,SL,Westangler Windenergie,,54° 37′ 20″ N,9° 38′ 3″ O,Sch-H


In [425]:
print(ho['Landkreis'].unique())

dict_ho={
        'OHSE':'OH'}

ho['Landkreis'].replace(dict_ho, inplace=True)
print(ho['Landkreis'].unique())

['NF' 'HEI' 'SL' 'OD' 'RD' 'IZ' 'FL' 'OH' 'SE' 'RZ' 'PI' 'HL' 'PLÖ' 'KI'
 'SG' 'OHSE']
['NF' 'HEI' 'SL' 'OD' 'RD' 'IZ' 'FL' 'OH' 'SE' 'RZ' 'PI' 'HL' 'PLÖ' 'KI'
 'SG']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/3320861172.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ho['Landkreis'].replace(dict_ho, inplace=True)


### Thuringen:

In [426]:
url_th="https://de.wikipedia.org/wiki/Liste_von_Windkraftanlagen_in_Thüringen"

# use requests.get() method with the provided static_url
response_th=requests.get(url_th).text
# assign the response to a object
soup_th=BeautifulSoup(response_th, 'html.parser')
# Use soup.title attribute
print('The title is:')
for title in soup_th.find_all('title'):
    print(title.get_text())
    

# Tabelle finden
html_tables = soup_th.find_all('table')
table_th = html_tables[0]  # Annahme: Die Tabelle ist die erste im HTML-Inhalt

# Liste für gefilterte Zeilen ohne durchgestrichenen Text
filtered_rows = []

# Durch alle Zeilen in der Tabelle iterieren
for row in table_th.find_all('tr'):
    # Überprüfen, ob die Zeile durchgestrichenen Text enthält
    td=row.find_all('td')
    has_struck_text = row.find(['s', 'del', 'strike'])

    if not has_struck_text:
        # Falls kein durchgestrichener Text enthalten ist, zur Liste hinzufügen
        row=[row.text for row in td]
        filtered_rows.append(row)  
print('The number of windparks in Thüringen is:', len(filtered_rows))


column_names = []
columns=table_th.find_all('th')
for column in columns:
    name=extract_column_from_header(column)
    
    if name!= None:
        if len(name)>0:
            column_names.append(name)
print(column_names) 


The title is:
Liste von Windkraftanlagen in Thüringen – Wikipedia
The number of windparks in Thüringen is: 103
['Name', 'Baujahr', 'Gesamt- leistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Land- kreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen']


In [427]:
# Initialisiere das Dictionary direkt mit leeren Listen als Werte
column_names = [
    'Name', 'Baujahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 
    'Ort', 'Landkreis', 'Koordinaten', 'Projektierer / Betreiber', 'Bemerkungen'
]

# Erstelle das Dictionary, wobei jedes Schlüsselpaar eine leere Liste ist
df = {column: [] for column in column_names}


#filling the dictionaries for the columns:

#filling the dictionary first for the year(Baujahr, Anfangjahr
for x in filtered_rows[1:973]:
    
    years=x[1].strip()
    years=''.join(c + ',' * (i in commas) for i, c in enumerate(years))
    df['Baujahr'].append(years)
    

df['Anfangjahr'] = [x[0:4] for x in df['Baujahr']]

#filling the dictionary for the remaining columns
for i in filtered_rows [1:973]:
    if i[0]!=None:
        name=i[0].strip()
        name=re.sub(r'\[\d+\]', '', name)
        df['Name'].append(name)
    
    df['Gesamtleistung (MW)'].append(i[2].strip())
    df['Anzahl'].append(i[3].strip())
    df['Typ (WKA)'].append(i[4].strip())
    df['Ort'].append(i[5].strip())
    df['Landkreis'].append(i[6].strip())
    df['Koordinaten'].append(i[7].strip())
    df['Projektierer / Betreiber'].append(i[8].strip())
    df['Bemerkungen'].append(i[-1].strip())                                             
  

df_th= pd.DataFrame({ key:pd.Series(value) for key, value in df.items() })

In [428]:
print(df_th.info())
#splitting the coordinates:
fn = lambda x: pd.Series([i for i in (x.split(','))])
coord = df_th['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
df_th['Latitude']=coord['Latitude']
df_th['Longitude']=coord['Longitude']

df_th['Land']='Th'
df_th['Anzahl'] = [int(str(i)) for i in df_th['Anzahl']]
df_th['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in df_th['Gesamtleistung (MW)']]


df_th.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Name                      102 non-null    object
 1   Baujahr                   102 non-null    object
 2   Gesamtleistung (MW)       102 non-null    object
 3   Anzahl                    102 non-null    object
 4   Typ (WKA)                 102 non-null    object
 5   Ort                       102 non-null    object
 6   Landkreis                 102 non-null    object
 7   Koordinaten               102 non-null    object
 8   Projektierer / Betreiber  102 non-null    object
 9   Bemerkungen               102 non-null    object
 10  Anfangjahr                102 non-null    object
dtypes: object(11)
memory usage: 8.9+ KB
None


,Name,Baujahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Koordinaten,Projektierer / Betreiber,Bemerkungen,Anfangjahr,Latitude,Longitude,Land
0,Windkraftanlage Breitungen/Werra,"1995,",0.60,1,Tacke TW 600e (1×),Breitungen-Winne,SM,"50° 45′ 22″ N, 10° 21′ 19″ O",,Sanierung 2010,1995,50° 45′ 22″ N,10° 21′ 19″ O,Th
1,Windkraftanlage Cretzschwitz,"2015,",2.35,1,Enercon E-92 (1×),Cretzschwitz,G,"50° 56′ 12″ N, 12° 6′ 24″ O",GoEn Planungsgesellschaft,,2015,50° 56′ 12″ N,12° 6′ 24″ O,Th
2,Windkraftanlage Diedorf,"1995,",0.60,1,Micon M 1500-600 (1×),Diedorf,UH,"51° 11′ 22″ N, 10° 17′ 32″ O",,,1995,51° 11′ 22″ N,10° 17′ 32″ O,Th


In [429]:
orts_th=df_th.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
orts_th.head(10)

,Anzahl
Landkreis,
SÖM,109
KYF,90
UH,85
WAK,83
ABG,64
GTHUH,64
SHK,58
EIC,57
AP,50


In [430]:
orts_th_MW=df_th.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
orts_th_MW.head(10)

,Gesamtleistung (MW)
Landkreis,
SÖM,310.95
KYF,219.85
UH,208.85
GTHUH,158.85
WAK,133.87
SHK,117.70
ABG,113.30
EIC,100.50
NDH,96.35


In [431]:
th=df_th[['Name', 'Baujahr', 'Anfangjahr', 'Gesamtleistung (MW)', 'Anzahl', 'Typ (WKA)', 'Ort', 'Landkreis', 'Projektierer / Betreiber', 'Bemerkungen', 'Latitude', 'Longitude', 'Land']]
th.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Windkraftanlage Breitungen/Werra,"1995,",1995,0.60,1,Tacke TW 600e (1×),Breitungen-Winne,SM,,Sanierung 2010,50° 45′ 22″ N,10° 21′ 19″ O,Th
1,Windkraftanlage Cretzschwitz,"2015,",2015,2.35,1,Enercon E-92 (1×),Cretzschwitz,G,GoEn Planungsgesellschaft,,50° 56′ 12″ N,12° 6′ 24″ O,Th
2,Windkraftanlage Diedorf,"1995,",1995,0.60,1,Micon M 1500-600 (1×),Diedorf,UH,,,51° 11′ 22″ N,10° 17′ 32″ O,Th


In [432]:
print(th['Landkreis'].unique())

dict_th={'SHKAP':'SHK',
        'KYFSÖM':'KYF',
        'KYFUH':'KYF',
        'APSLF':'AP',
        'WAKGTH':'WAK',
        'GTHUH':'GTH',
        'SÖ':'SÖM'}

th['Landkreis'].replace(dict_th, inplace=True)
print(th['Landkreis'].unique())

['SM' 'G' 'UH' 'WAK' 'EIC' 'SOK' 'SHK' 'GRZ' 'SÖ' 'ABG' 'SLF' 'HBN' 'SÖM'
 'AP' 'GTH' 'KYF' 'SHKAP' 'EF' 'KYFSÖM' 'NDH' 'KYFUH' 'APSLF' 'WAKGTH'
 'GTHUH' 'IK']
['SM' 'G' 'UH' 'WAK' 'EIC' 'SOK' 'SHK' 'GRZ' 'SÖM' 'ABG' 'SLF' 'HBN' 'AP'
 'GTH' 'KYF' 'EF' 'NDH' 'IK']


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/3800136383.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th['Landkreis'].replace(dict_th, inplace=True)


In [479]:
landes_list=[bw, th, ho, an, s , sa, rp, nw, bayern, mvp, hbh, he, br]
landes=pd.concat(landes_list)
landes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4277 entries, 0 to 289
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name                      4277 non-null   object 
 1   Baujahr                   4277 non-null   object 
 2   Anfangjahr                4277 non-null   object 
 3   Gesamtleistung (MW)       4277 non-null   float64
 4   Anzahl                    4277 non-null   int64  
 5   Typ (WKA)                 4277 non-null   object 
 6   Ort                       4277 non-null   object 
 7   Landkreis                 4277 non-null   object 
 8   Projektierer / Betreiber  4277 non-null   object 
 9   Bemerkungen               4277 non-null   object 
 10  Latitude                  4277 non-null   object 
 11  Longitude                 4277 non-null   object 
 12  Land                      4277 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 467.8+ KB


In [480]:
landes.head(3)

,Name,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land
0,Forschungswindkraftanlage RedoxWind,"2017,",2017,2.0,1,Qreon Q82 (1×),Karlsruhe,KA,Fraunhofer ICT,Forschungsprojekt mit kombiniertem Batteriespe...,49° 1′ 9″ N,8° 31′ 4″ O,BW
1,Windkraftanlage Alexanderschanze,"2003,",2003,1.5,1,Südwind S70 (1×),Bad Peterstal-Griesbach,OG,"Schmalz, Windenergieanlage „Bei der Schanz“",errichtet an der Schwarzwaldhochstraße im Bere...,48° 28′ 22″ N,8° 16′ 7″ O,BW
2,Windkraftanlage Alpirsbach,"1999,2013,",1999,2.3,1,Enercon E-70 E4 (1×),Römlinsdorf,FDS,"Windkraft Römlinsdorf, Stadtwerke Stuttgart",Repowering 2013 (1× Enercon E-70 E4 statt 1× F...,48° 20′ 0″ N,8° 28′ 34″ O,BW


In [481]:
data=pd.read_csv('Landkreis_ID.csv', index_col=False)
data.head(5)

,Code,NUTS 1.1,NUTS 2.1,Name,Landkreis
0,DEB12,Rheinland-Pfalz,Koblenz,Ahrweiler,AW
1,DE275,Bayern,Schwaben,Aichach-Friedberg,AIC
2,DE145,Baden-Württemberg,Tübingen,Alb-Donau-Kreis,UL
3,DEG0M,Thüringen,Thüringen,Altenburger Land,ABG
4,DEB13,Rheinland-Pfalz,Koblenz,Altenkirchen (Westerwald),AK


In [482]:
#data.to_csv('Windanlage_DE.csv')

In [483]:
new_data=pd.merge(landes,data, how='left', on='Landkreis')

In [484]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4667 entries, 0 to 4666
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name_x                    4667 non-null   object 
 1   Baujahr                   4667 non-null   object 
 2   Anfangjahr                4667 non-null   object 
 3   Gesamtleistung (MW)       4667 non-null   float64
 4   Anzahl                    4667 non-null   int64  
 5   Typ (WKA)                 4667 non-null   object 
 6   Ort                       4667 non-null   object 
 7   Landkreis                 4667 non-null   object 
 8   Projektierer / Betreiber  4667 non-null   object 
 9   Bemerkungen               4667 non-null   object 
 10  Latitude                  4667 non-null   object 
 11  Longitude                 4667 non-null   object 
 12  Land                      4667 non-null   object 
 13  Code                      4615 non-null   object 
 14  NUTS 1.1

In [478]:
df_nan=new_data[new_data['Code'].isnull()]
df_nan

,Name_x,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land,Code,NUTS 1.1,NUTS 2.1,Name_y
1152,Windkraftanlage Riesweiler,"1999,",1999,0.250,1,Nordex N29/250 (1×),Riesweiler,HOM,,,49° 8′ 3″ N,7° 18′ 22″ O,SA,NaN,NaN,NaN,NaN
1164,Windpark Auf der weißen Trisch (Windpark Kirrb...,"2017,",2017,9.600,4,Nordex N117/2400 (4×),Kirrberg,HOM,EnBW Energie Baden-Württemberg AG,,49° 17′ 12″ N,7° 21′ 22″ O,SA,NaN,NaN,NaN,NaN
1196,Windpark Webenheim,"2016,",2016,6.150,3,Senvion MM92 (3×),Webenheim,HOM,EnBW Energie Baden-Württemberg AG,,49° 15′ 1″ N,7° 17′ 17″ O,SA,NaN,NaN,NaN,NaN
2843,Windkraftanlage Dalkendorf,"1999,",1999,0.660,1,Vestas V47-660kW (1×),Dalkendorf,LRO,,,53° 49′ 11″ N,12° 30′ 1″ O,MP,NaN,NaN,NaN,NaN
2854,Windkraftanlage Klein Sien,?,?,0.500,1,Enercon E-40/5.40 (1×),Jürgenshagen,LRO,,,53° 53′ 18″ N,11° 48′ 47″ O,MP,NaN,NaN,NaN,NaN
2861,Windkraftanlage Mönchhagen,"1994,",1994,0.500,1,Vestas V39-500kW (1×),Mönchhagen,LRO,,"früher zwei Anlagen, eine wurde mittlerweile w...",54° 9′ 16″ N,12° 13′ 9″ O,MP,NaN,NaN,NaN,NaN
2867,Windkraftanlage Radegast,"1996,",1996,0.500,1,Nordtank NTK 500/41 (1×),Satow,LRO,,,53° 57′ 29″ N,11° 50′ 31″ O,MP,NaN,NaN,NaN,NaN
2872,Windkraftanlage Stubbendorf,"2020,",2020,0.030,1,Südwind N 1230 (1×),Stubbendorf,LRO,Gesellschaft zur Förderung der Europäischen Ko...,,53° 57′ 15″ N,12° 48′ 4″ O,MP,NaN,NaN,NaN,NaN
2881,Windkraftanlage Ziesendorf,"1996,",1996,0.225,1,Vestas V29-225kW (1×),Ziesendorf,LRO,WIND-projekt,,54° 0′ 7″ N,12° 2′ 0″ O,MP,NaN,NaN,NaN,NaN
2883,Windkraftanlagen Bülow,"1994,",1994,1.000,2,Enercon E-40/5.40 (2×),Gutow,LRO,,,53° 46′ 5″ N,12° 6′ 43″ O,MP,NaN,NaN,NaN,NaN


In [485]:
new_data['Code'].loc[new_data['Landkreis']=='CB']='DE402'
new_data['Code'].loc[new_data['Landkreis']=='HOM']='DEC05'
new_data['NUTS 1.1'].loc[new_data['Landkreis']=='CB']='DE4'
new_data['NUTS 1.1'].loc[new_data['Landkreis']=='HOM']='DEC'

new_data['NUTS 2.1'].loc[new_data['Landkreis']=='CB']='DE40'
new_data['NUTS 2.1'].loc[new_data['Landkreis']=='HOM']='DEC0'
new_data['Name_y'].loc[new_data['Landkreis']=='CB']='Cottbus, Kreisfreie Stadt'
new_data['Name_y'].loc[new_data['Landkreis']=='HOM']='Saarpfalz-Kreis'

new_data['Code'].loc[new_data['Landkreis']=='LRO']='DE80K'
new_data['Code'].loc[new_data['Landkreis']=='BH']='DE501'
new_data['NUTS 1.1'].loc[new_data['Landkreis']=='LRO']='DE8'
new_data['NUTS 1.1'].loc[new_data['Landkreis']=='BH']='DE5'

new_data['NUTS 2.1'].loc[new_data['Landkreis']=='LRO']='DE80'
new_data['NUTS 2.1'].loc[new_data['Landkreis']=='BH']='DE50'
new_data['Name_y'].loc[new_data['Landkreis']=='LRO']='Rostock, Landkreis'
new_data['Name_y'].loc[new_data['Landkreis']=='BH']='Bremen, Kreisfreie Stadt'


/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/228946282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Code'].loc[new_data['Landkreis']=='CB']='DE402'
/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/228946282.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Code'].loc[new_data['Landkreis']=='HOM']='DEC05'
/var/folders/bg/6h6fxpzx7_v6jgvd0c31r_7m0000gn/T/ipykernel_48949/228946282.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [486]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4667 entries, 0 to 4666
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name_x                    4667 non-null   object 
 1   Baujahr                   4667 non-null   object 
 2   Anfangjahr                4667 non-null   object 
 3   Gesamtleistung (MW)       4667 non-null   float64
 4   Anzahl                    4667 non-null   int64  
 5   Typ (WKA)                 4667 non-null   object 
 6   Ort                       4667 non-null   object 
 7   Landkreis                 4667 non-null   object 
 8   Projektierer / Betreiber  4667 non-null   object 
 9   Bemerkungen               4667 non-null   object 
 10  Latitude                  4667 non-null   object 
 11  Longitude                 4667 non-null   object 
 12  Land                      4667 non-null   object 
 13  Code                      4667 non-null   object 
 14  NUTS 1.1

In [491]:
# Drop duplicate rows
new_data = new_data.drop_duplicates(subset=['Name_x'])
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4251 entries, 0 to 4666
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name_x                    4251 non-null   object 
 1   Baujahr                   4251 non-null   object 
 2   Anfangjahr                4251 non-null   object 
 3   Gesamtleistung (MW)       4251 non-null   float64
 4   Anzahl                    4251 non-null   int64  
 5   Typ (WKA)                 4251 non-null   object 
 6   Ort                       4251 non-null   object 
 7   Landkreis                 4251 non-null   object 
 8   Projektierer / Betreiber  4251 non-null   object 
 9   Bemerkungen               4251 non-null   object 
 10  Latitude                  4251 non-null   object 
 11  Longitude                 4251 non-null   object 
 12  Land                      4251 non-null   object 
 13  Code                      4251 non-null   object 
 14  NUTS 1.1     

In [492]:
new_data['Anzahl'] = [int(str(i)) for i in new_data['Anzahl']]
new_data['Gesamtleistung (MW)'] = [float(str(i).replace(",", ".")) for i in new_data['Gesamtleistung (MW)']]



new_data_numbers=new_data.groupby(['Landkreis']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
new_data_numbers.head(10)

,Anzahl
Landkreis,
NF,861
HEI,852
UM,659
PR,556
PB,548
SL,510
CUX,502
AUR,495
EL,489


In [493]:
new_data_mw=new_data.groupby(['Landkreis']).agg({'Gesamtleistung (MW)':'sum'}).sort_values(by='Gesamtleistung (MW)',ascending=False)
new_data_mw.head(10)

,Gesamtleistung (MW)
Landkreis,
NF,2508.940
HEI,2361.500
UM,1697.010
SL,1440.400
PB,1409.985
EL,1280.580
PR,1215.010
DH,941.555
OH,916.675


In [494]:
new_data['Latitude'] = new_data['Latitude'].str.split('N', n=1).str.get(0)
new_data['Longitude'] = new_data['Longitude'].str.split('O', n=1).str.get(0)
new_data.head()

,Name_x,Baujahr,Anfangjahr,Gesamtleistung (MW),Anzahl,Typ (WKA),Ort,Landkreis,Projektierer / Betreiber,Bemerkungen,Latitude,Longitude,Land,Code,NUTS 1.1,NUTS 2.1,Name_y
0,Forschungswindkraftanlage RedoxWind,"2017,",2017,2.000,1,Qreon Q82 (1×),Karlsruhe,KA,Fraunhofer ICT,Forschungsprojekt mit kombiniertem Batteriespe...,49° 1′ 9″,8° 31′ 4″,BW,DE123,Baden-Württemberg,Karlsruhe,"Karlsruhe, Landkreis"
2,Windkraftanlage Alexanderschanze,"2003,",2003,1.500,1,Südwind S70 (1×),Bad Peterstal-Griesbach,OG,"Schmalz, Windenergieanlage „Bei der Schanz“",errichtet an der Schwarzwaldhochstraße im Bere...,48° 28′ 22″,8° 16′ 7″,BW,DE134,Baden-Württemberg,Freiburg,Ortenaukreis
3,Windkraftanlage Alpirsbach,"1999,2013,",1999,2.300,1,Enercon E-70 E4 (1×),Römlinsdorf,FDS,"Windkraft Römlinsdorf, Stadtwerke Stuttgart",Repowering 2013 (1× Enercon E-70 E4 statt 1× F...,48° 20′ 0″,8° 28′ 34″,BW,DE12C,Baden-Württemberg,Karlsruhe,Freudenstadt
4,Windkraftanlage Auenberg,"1990,",1990,0.095,1,Reymo DANmark 22 (1×),Hausen vor Wald,VS,Pionier-Windmühle Auenberg,(*); erste kommerziell betriebene Windkraftanl...,47° 53′ 19″,8° 28′ 24″,BW,DE136,Baden-Württemberg,Freiburg,Schwarzwald-Baar-Kreis
5,Windkraftanlage auf dem Hohwartsattel,"1992,",1992,0.300,1,Enercon E-33 (1×),Breitnau,FR,"Christian Riesterer, Uwe Tischler",(*),47° 56′ 14″,8° 3′ 11″,BW,DE132,Baden-Württemberg,Freiburg,Breisgau-Hochschwarzwald


In [497]:
new_data_coord=new_data.groupby(['Latitude','Longitude', 'Landkreis', 'Name_y', 'Code', 'Land']).agg({'Anzahl':'sum'}).sort_values(by='Anzahl',ascending=False)
new_data_coord.head(10)

,,,,,,Anzahl
Latitude,Longitude,Landkreis,Name_y,Code,Land,
53° 36′ 37″,7° 25′ 45″,WTM,Wittmund,DE94H,"HB,HH, H",144
53° 58′ 26″,8° 56′ 0″,HEI,Dithmarschen,DEF05,Sch-H,136
52° 0′ 3″,12° 49′ 31″,PM,"Potsdam, Kreisfreie Stadt",DE404,Ber Br,114
51° 52′ 34″,13° 31′ 15″,TF,Teltow-Fläming,DE40H,Ber Br,114
53° 21′ 24″,13° 56′ 0″,UM,Uckermark,DE40I,Ber Br,113
54° 36′ 24″,8° 54′ 8″,NF,Nordfriesland,DEF07,Sch-H,88
51.638056°,8.915°,PB,Paderborn,DEA47,NW,85
51° 7′ 32″,11° 57′ 31″,BLK,Burgenlandkreis,DEE08,SA,83
52° 32′ 14″,12° 52′ 11″,HVL,Havelland,DE408,Ber Br,82


In [498]:
new_data.to_csv('Landkreis_id_windanlage.csv')

In [499]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4251 entries, 0 to 4666
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name_x                    4251 non-null   object 
 1   Baujahr                   4251 non-null   object 
 2   Anfangjahr                4251 non-null   object 
 3   Gesamtleistung (MW)       4251 non-null   float64
 4   Anzahl                    4251 non-null   int64  
 5   Typ (WKA)                 4251 non-null   object 
 6   Ort                       4251 non-null   object 
 7   Landkreis                 4251 non-null   object 
 8   Projektierer / Betreiber  4251 non-null   object 
 9   Bemerkungen               4251 non-null   object 
 10  Latitude                  4251 non-null   object 
 11  Longitude                 4251 non-null   object 
 12  Land                      4251 non-null   object 
 13  Code                      4251 non-null   object 
 14  NUTS 1.1     